In [ ]:
## This file implements neural networks before and after lasso selection for p0017Skpresabs_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [3]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [4]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0017Skpresabs_qual.csv')
df.shape

(253, 103)

In [5]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [6]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    1
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [7]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTAATACATAT,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA,...,AGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTTT,AGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTTTAATTT,AGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCT,AGATATTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTT,ACTTTCGAATT,AATTAAAATCTTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTT,AATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTT,AATGGTATGCCTTTGTTTGTATAGTTTTCACTTCCACCTTTGGGAGTCTTTCCACTACCTATTTTGGTAGTAAGATTCCCTAACTTCTTCTCTTCCCATT,AAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTT,pheno
0,107,1,0,1,0,0,1,1,0,1,...,1,1,1,1,0,1,1,1,1,0
1,109,1,0,1,0,0,1,1,0,1,...,1,1,1,1,0,1,1,1,1,0
2,115,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,120335,1,0,1,0,0,1,1,0,1,...,1,1,1,1,0,1,1,1,1,0
4,120337,1,0,1,0,0,1,1,0,1,...,1,1,1,1,0,1,1,1,1,0


In [8]:
df['pheno'].value_counts()

0    216
1     35
2      2
Name: pheno, dtype: int64

In [9]:
df_clean = df.drop(columns=['id'])

In [10]:
df_clean.shape

(253, 102)

In [11]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTAATACATAT,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA,TTCAAGAAGGAGA,...,AGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTTT,AGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTTTAATTT,AGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCT,AGATATTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTT,ACTTTCGAATT,AATTAAAATCTTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTT,AATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTT,AATGGTATGCCTTTGTTTGTATAGTTTTCACTTCCACCTTTGGGAGTCTTTCCACTACCTATTTTGGTAGTAAGATTCCCTAACTTCTTCTCTTCCCATT,AAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAGTAATCTTAATGCTTCAAAAGATTT,pheno
0,1,0,1,0,0,1,1,0,1,0,...,1,1,1,1,0,1,1,1,1,0
1,1,0,1,0,0,1,1,0,1,0,...,1,1,1,1,0,1,1,1,1,0
2,1,1,1,0,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
3,1,0,1,0,0,1,1,0,1,0,...,1,1,1,1,0,1,1,1,1,0
4,1,0,1,0,0,1,1,0,1,0,...,1,1,1,1,0,1,1,1,1,0


In [12]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 102) (253,)


In [13]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 216), (1, 216), (2, 216)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [14]:
############# Fully-Connected Neural Network ################

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [16]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [17]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [18]:
dat

,0,test
0,312,1
1,CFBRSa27,0
2,BCH-SA-01,1
3,GA27,1
4,NRS209,2
...,...,...
190,NRS209,2
191,NRS235,0
192,NRS240,1
193,NRS110,2


In [19]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [20]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 315us/step - loss: 0.8689 - accuracy: 0.6358 - val_loss: 0.7299 - val_accuracy: 0.7077
Epoch 2/1000
453/453 [==============================] - 0s 179us/step - loss: 0.6227 - accuracy: 0.7991 - val_loss: 0.5903 - val_accuracy: 0.7231
Epoch 3/1000
453/453 [==============================] - 0s 213us/step - loss: 0.5069 - accuracy: 0.8057 - val_loss: 0.5201 - val_accuracy: 0.7590
Epoch 4/1000
453/453 [==============================] - 0s 147us/step - loss: 0.4500 - accuracy: 0.8057 - val_loss: 0.4899 - val_accuracy: 0.7538
Epoch 5/1000
453/453 [==============================] - 0s 145us/step - loss: 0.4057 - accuracy: 0.8124 - val_loss: 0.4994 - val_accuracy: 0.7590
Epoch 6/1000
453/453 [==============================] - 0s 136us/step - loss: 0.3910 - accuracy: 0.8389 - val_loss: 0.4689 - val_accuracy: 0.7590
Epoch 7/1000
453/453 [==============================] - 0s 225us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2431 - accuracy: 0.8786 - val_loss: 0.3950 - val_accuracy: 0.7949
Epoch 113/1000
453/453 [==============================] - 0s 97us/step - loss: 0.2483 - accuracy: 0.8786 - val_loss: 0.3900 - val_accuracy: 0.7949
Epoch 114/1000
453/453 [==============================] - 0s 91us/step - loss: 0.2519 - accuracy: 0.8808 - val_loss: 0.3786 - val_accuracy: 0.8154
Epoch 115/1000
453/453 [==============================] - 0s 90us/step - loss: 0.2506 - accuracy: 0.8808 - val_loss: 0.3750 - val_accuracy: 0.8410
Epoch 116/1000
453/453 [==============================] - 0s 90us/step - loss: 0.2602 - accuracy: 0.8852 - val_loss: 0.3733 - val_accuracy: 0.8410
Epoch 117/1000
453/453 [==============================] - 0s 91us/step - loss: 0.2519 - accuracy: 0.8830 - val_loss: 0.3808 - val_accuracy: 0.8256
Epoch 118/1000
453/453 [==============================] - 0s 92us/step - loss: 0.2498 - accuracy: 0.8830 - val_loss: 

453/453 [==============================] - 0s 96us/step - loss: 0.2309 - accuracy: 0.9029 - val_loss: 0.4021 - val_accuracy: 0.8256
Epoch 223/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2362 - accuracy: 0.9029 - val_loss: 0.3735 - val_accuracy: 0.8308
Epoch 224/1000
453/453 [==============================] - 0s 148us/step - loss: 0.2273 - accuracy: 0.8985 - val_loss: 0.3758 - val_accuracy: 0.8308
Epoch 225/1000
453/453 [==============================] - 0s 140us/step - loss: 0.2416 - accuracy: 0.8985 - val_loss: 0.3805 - val_accuracy: 0.8308
Epoch 226/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2348 - accuracy: 0.9029 - val_loss: 0.3930 - val_accuracy: 0.8308
Epoch 227/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2285 - accuracy: 0.9029 - val_loss: 0.3880 - val_accuracy: 0.8205
Epoch 228/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2377 - accuracy: 0.8896 - val_loss: 0.3763 - v

453/453 [==============================] - 0s 136us/step - loss: 0.2168 - accuracy: 0.9051 - val_loss: 0.3899 - val_accuracy: 0.8308
Epoch 333/1000
453/453 [==============================] - 0s 181us/step - loss: 0.2224 - accuracy: 0.9073 - val_loss: 0.3971 - val_accuracy: 0.8308
Epoch 334/1000
453/453 [==============================] - 0s 177us/step - loss: 0.2184 - accuracy: 0.9073 - val_loss: 0.4049 - val_accuracy: 0.8308
Epoch 335/1000
453/453 [==============================] - 0s 190us/step - loss: 0.2221 - accuracy: 0.9051 - val_loss: 0.3972 - val_accuracy: 0.8308
Epoch 336/1000
453/453 [==============================] - 0s 165us/step - loss: 0.2219 - accuracy: 0.9029 - val_loss: 0.3961 - val_accuracy: 0.8308
Epoch 337/1000
453/453 [==============================] - 0s 167us/step - loss: 0.2166 - accuracy: 0.9073 - val_loss: 0.4049 - val_accuracy: 0.8308
Epoch 338/1000
453/453 [==============================] - 0s 176us/step - loss: 0.2175 - accuracy: 0.9073 - val_loss: 0.3990 - 

453/453 [==============================] - 0s 107us/step - loss: 0.2123 - accuracy: 0.9073 - val_loss: 0.4172 - val_accuracy: 0.8410
Epoch 443/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2219 - accuracy: 0.9029 - val_loss: 0.4274 - val_accuracy: 0.8308
Epoch 444/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2164 - accuracy: 0.9073 - val_loss: 0.4241 - val_accuracy: 0.8308
Epoch 445/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2146 - accuracy: 0.9073 - val_loss: 0.4239 - val_accuracy: 0.8308
Epoch 446/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2135 - accuracy: 0.9073 - val_loss: 0.4252 - val_accuracy: 0.8308
Epoch 447/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2118 - accuracy: 0.9073 - val_loss: 0.4246 - val_accuracy: 0.8308
Epoch 448/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2194 - accuracy: 0.9073 - val_loss: 0.4242 - 

453/453 [==============================] - 0s 116us/step - loss: 0.2200 - accuracy: 0.9073 - val_loss: 0.4587 - val_accuracy: 0.8308
Epoch 553/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2137 - accuracy: 0.9073 - val_loss: 0.4408 - val_accuracy: 0.8308
Epoch 554/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2123 - accuracy: 0.9073 - val_loss: 0.4459 - val_accuracy: 0.8308
Epoch 555/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2101 - accuracy: 0.9073 - val_loss: 0.4480 - val_accuracy: 0.8308
Epoch 556/1000
453/453 [==============================] - 0s 146us/step - loss: 0.2141 - accuracy: 0.9073 - val_loss: 0.4566 - val_accuracy: 0.8308
Epoch 557/1000
453/453 [==============================] - 0s 132us/step - loss: 0.2130 - accuracy: 0.9051 - val_loss: 0.4480 - val_accuracy: 0.8308
Epoch 558/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2158 - accuracy: 0.9073 - val_loss: 0.4536 - 

453/453 [==============================] - 0s 122us/step - loss: 0.2129 - accuracy: 0.9073 - val_loss: 0.4709 - val_accuracy: 0.8308
Epoch 663/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2102 - accuracy: 0.9073 - val_loss: 0.4717 - val_accuracy: 0.8308
Epoch 664/1000
453/453 [==============================] - 0s 140us/step - loss: 0.2099 - accuracy: 0.9073 - val_loss: 0.4761 - val_accuracy: 0.8308
Epoch 665/1000
453/453 [==============================] - 0s 174us/step - loss: 0.2113 - accuracy: 0.9051 - val_loss: 0.4700 - val_accuracy: 0.8308
Epoch 666/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2110 - accuracy: 0.9073 - val_loss: 0.4806 - val_accuracy: 0.8308
Epoch 667/1000
453/453 [==============================] - 0s 242us/step - loss: 0.2093 - accuracy: 0.9073 - val_loss: 0.4772 - val_accuracy: 0.8308
Epoch 668/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2097 - accuracy: 0.9073 - val_loss: 0.4764 - 

453/453 [==============================] - 0s 126us/step - loss: 0.2133 - accuracy: 0.9051 - val_loss: 0.4789 - val_accuracy: 0.8308
Epoch 773/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2147 - accuracy: 0.9029 - val_loss: 0.4956 - val_accuracy: 0.8308
Epoch 774/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2121 - accuracy: 0.9073 - val_loss: 0.4873 - val_accuracy: 0.8308
Epoch 775/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2108 - accuracy: 0.9073 - val_loss: 0.4915 - val_accuracy: 0.8308
Epoch 776/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2153 - accuracy: 0.9073 - val_loss: 0.4849 - val_accuracy: 0.8308
Epoch 777/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2138 - accuracy: 0.9051 - val_loss: 0.4868 - val_accuracy: 0.8308
Epoch 778/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2091 - accuracy: 0.9073 - val_loss: 0.4896 - 

453/453 [==============================] - 0s 124us/step - loss: 0.2100 - accuracy: 0.9073 - val_loss: 0.4832 - val_accuracy: 0.8308
Epoch 883/1000
453/453 [==============================] - 0s 144us/step - loss: 0.2099 - accuracy: 0.9073 - val_loss: 0.4908 - val_accuracy: 0.8308
Epoch 884/1000
453/453 [==============================] - 0s 139us/step - loss: 0.2141 - accuracy: 0.9073 - val_loss: 0.4975 - val_accuracy: 0.8308
Epoch 885/1000
453/453 [==============================] - 0s 184us/step - loss: 0.2123 - accuracy: 0.9073 - val_loss: 0.4950 - val_accuracy: 0.8308
Epoch 886/1000
453/453 [==============================] - 0s 139us/step - loss: 0.2094 - accuracy: 0.9073 - val_loss: 0.4933 - val_accuracy: 0.8308
Epoch 887/1000
453/453 [==============================] - 0s 167us/step - loss: 0.2114 - accuracy: 0.9073 - val_loss: 0.4964 - val_accuracy: 0.8308
Epoch 888/1000
453/453 [==============================] - 0s 175us/step - loss: 0.2136 - accuracy: 0.9073 - val_loss: 0.5038 - 

453/453 [==============================] - 0s 110us/step - loss: 0.2106 - accuracy: 0.9073 - val_loss: 0.4873 - val_accuracy: 0.8308
Epoch 993/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2122 - accuracy: 0.9073 - val_loss: 0.4924 - val_accuracy: 0.8308
Epoch 994/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2091 - accuracy: 0.9073 - val_loss: 0.4934 - val_accuracy: 0.8308
Epoch 995/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2112 - accuracy: 0.9073 - val_loss: 0.4950 - val_accuracy: 0.8308
Epoch 996/1000
453/453 [==============================] - 0s 134us/step - loss: 0.2115 - accuracy: 0.9073 - val_loss: 0.5055 - val_accuracy: 0.8308
Epoch 997/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2123 - accuracy: 0.9073 - val_loss: 0.4960 - val_accuracy: 0.8308
Epoch 998/1000
453/453 [==============================] - 0s 142us/step - loss: 0.2134 - accuracy: 0.9073 - val_loss: 0.5069 - 

In [33]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

195/195 [==============================] - 0s 58us/step
over-sampling test accuracy: 83.08%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 0, 1, 1, 2, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 1, 2, 1, 0,
       2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 1, 2, 2, 2, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 0, 2,
       1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 1, 1, 1, 1, 2, 1, 2,
       2, 1, 0, 1, 0, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, 1, 1, 0, 0, 2, 0, 2,
       0, 0, 1, 2, 2, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 0, 0, 2,
       0, 2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2, 1, 0, 2, 0, 1, 2, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,312,1,0
1,CFBRSa27,0,0
2,BCH-SA-01,1,1
3,GA27,1,1
4,NRS209,2,2
...,...,...,...
190,NRS209,2,2
191,NRS235,0,0
192,NRS240,1,1
193,NRS110,2,2


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,7.352306e-01,2.647694e-01,2.742082e-10
1,7.352306e-01,2.647694e-01,2.742082e-10
2,1.729426e-01,8.270574e-01,5.515762e-12
3,2.531625e-01,7.468375e-01,1.100956e-10
4,5.784774e-09,9.270067e-09,1.000000e+00
...,...,...,...
190,5.784774e-09,9.270067e-09,1.000000e+00
191,9.986609e-01,1.339073e-03,2.859718e-11
192,5.067671e-04,9.994932e-01,4.060507e-09
193,1.642337e-08,1.409822e-08,1.000000e+00


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p17s.csv", index = False,
         header=None)

In [37]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 177us/step - loss: 0.2117 - accuracy: 0.9073 - val_loss: 0.4137 - val_accuracy: 0.8308
Epoch 2/1000
453/453 [==============================] - 0s 143us/step - loss: 0.2109 - accuracy: 0.9073 - val_loss: 0.4066 - val_accuracy: 0.8308
Epoch 3/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2096 - accuracy: 0.9073 - val_loss: 0.4070 - val_accuracy: 0.8308
Epoch 4/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2081 - accuracy: 0.9073 - val_loss: 0.4041 - val_accuracy: 0.8308
Epoch 5/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2088 - accuracy: 0.9073 - val_loss: 0.4069 - val_accuracy: 0.8308
Epoch 6/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2113 - accuracy: 0.9073 - val_loss: 0.4113 - val_accuracy: 0.8308
Epoch 7/1000
453/453 [==============================] - 0s 103us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 100us/step - loss: 0.2109 - accuracy: 0.9073 - val_loss: 0.4270 - val_accuracy: 0.8308
Epoch 113/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2104 - accuracy: 0.9073 - val_loss: 0.4294 - val_accuracy: 0.8308
Epoch 114/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2109 - accuracy: 0.9073 - val_loss: 0.4216 - val_accuracy: 0.8308
Epoch 115/1000
453/453 [==============================] - 0s 103us/step - loss: 0.2118 - accuracy: 0.9073 - val_loss: 0.4211 - val_accuracy: 0.8308
Epoch 116/1000
453/453 [==============================] - 0s 100us/step - loss: 0.2081 - accuracy: 0.9073 - val_loss: 0.4157 - val_accuracy: 0.8308
Epoch 117/1000
453/453 [==============================] - 0s 100us/step - loss: 0.2091 - accuracy: 0.9073 - val_loss: 0.4193 - val_accuracy: 0.8308
Epoch 118/1000
453/453 [==============================] - 0s 97us/step - loss: 0.2082 - accuracy: 0.9073 - val_l

453/453 [==============================] - 0s 350us/step - loss: 0.2089 - accuracy: 0.9073 - val_loss: 0.4274 - val_accuracy: 0.8308
Epoch 223/1000
453/453 [==============================] - 0s 208us/step - loss: 0.2135 - accuracy: 0.9073 - val_loss: 0.4306 - val_accuracy: 0.8308
Epoch 224/1000
453/453 [==============================] - 0s 225us/step - loss: 0.2098 - accuracy: 0.9073 - val_loss: 0.4348 - val_accuracy: 0.8308
Epoch 225/1000
453/453 [==============================] - 0s 226us/step - loss: 0.2141 - accuracy: 0.9073 - val_loss: 0.4369 - val_accuracy: 0.8308
Epoch 226/1000
453/453 [==============================] - 0s 190us/step - loss: 0.2084 - accuracy: 0.9073 - val_loss: 0.4352 - val_accuracy: 0.8308
Epoch 227/1000
453/453 [==============================] - 0s 175us/step - loss: 0.2101 - accuracy: 0.9073 - val_loss: 0.4273 - val_accuracy: 0.8308
Epoch 228/1000
453/453 [==============================] - 0s 185us/step - loss: 0.2088 - accuracy: 0.9073 - val_loss: 0.4352 - 

453/453 [==============================] - 0s 111us/step - loss: 0.2088 - accuracy: 0.9073 - val_loss: 0.4163 - val_accuracy: 0.8308
Epoch 333/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2100 - accuracy: 0.9051 - val_loss: 0.4333 - val_accuracy: 0.8308
Epoch 334/1000
453/453 [==============================] - 0s 146us/step - loss: 0.2092 - accuracy: 0.9073 - val_loss: 0.4333 - val_accuracy: 0.8308
Epoch 335/1000
453/453 [==============================] - 0s 199us/step - loss: 0.2095 - accuracy: 0.9073 - val_loss: 0.4199 - val_accuracy: 0.8308
Epoch 336/1000
453/453 [==============================] - 0s 138us/step - loss: 0.2093 - accuracy: 0.9073 - val_loss: 0.4240 - val_accuracy: 0.8308
Epoch 337/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2127 - accuracy: 0.9073 - val_loss: 0.4252 - val_accuracy: 0.8308
Epoch 338/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2071 - accuracy: 0.9073 - val_loss: 0.4183 - 

453/453 [==============================] - 0s 107us/step - loss: 0.2084 - accuracy: 0.9073 - val_loss: 0.4167 - val_accuracy: 0.8308
Epoch 443/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2088 - accuracy: 0.9073 - val_loss: 0.4164 - val_accuracy: 0.8308
Epoch 444/1000
453/453 [==============================] - 0s 137us/step - loss: 0.2076 - accuracy: 0.9073 - val_loss: 0.4160 - val_accuracy: 0.8308
Epoch 445/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2105 - accuracy: 0.9073 - val_loss: 0.4394 - val_accuracy: 0.8308
Epoch 446/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2071 - accuracy: 0.9073 - val_loss: 0.4297 - val_accuracy: 0.8308
Epoch 447/1000
453/453 [==============================] - 0s 137us/step - loss: 0.2088 - accuracy: 0.9073 - val_loss: 0.4311 - val_accuracy: 0.8308
Epoch 448/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2080 - accuracy: 0.9073 - val_loss: 0.4173 - 

453/453 [==============================] - 0s 112us/step - loss: 0.2095 - accuracy: 0.9073 - val_loss: 0.4259 - val_accuracy: 0.8308
Epoch 553/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2079 - accuracy: 0.9073 - val_loss: 0.4267 - val_accuracy: 0.8308
Epoch 554/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2080 - accuracy: 0.9073 - val_loss: 0.4220 - val_accuracy: 0.8308
Epoch 555/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2095 - accuracy: 0.9073 - val_loss: 0.4258 - val_accuracy: 0.8308
Epoch 556/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2063 - accuracy: 0.9073 - val_loss: 0.4216 - val_accuracy: 0.8308
Epoch 557/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2097 - accuracy: 0.9073 - val_loss: 0.4186 - val_accuracy: 0.8308
Epoch 558/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2075 - accuracy: 0.9073 - val_loss: 0.4174 - 

453/453 [==============================] - 0s 112us/step - loss: 0.2098 - accuracy: 0.9073 - val_loss: 0.4114 - val_accuracy: 0.8308
Epoch 663/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2108 - accuracy: 0.9073 - val_loss: 0.4406 - val_accuracy: 0.8308
Epoch 664/1000
453/453 [==============================] - 0s 169us/step - loss: 0.2100 - accuracy: 0.9073 - val_loss: 0.4329 - val_accuracy: 0.8308
Epoch 665/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2089 - accuracy: 0.9073 - val_loss: 0.4317 - val_accuracy: 0.8308
Epoch 666/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2075 - accuracy: 0.9073 - val_loss: 0.4254 - val_accuracy: 0.8308
Epoch 667/1000
453/453 [==============================] - 0s 139us/step - loss: 0.2087 - accuracy: 0.9073 - val_loss: 0.4196 - val_accuracy: 0.8308
Epoch 668/1000
453/453 [==============================] - 0s 155us/step - loss: 0.2068 - accuracy: 0.9073 - val_loss: 0.4131 - 

453/453 [==============================] - 0s 100us/step - loss: 0.2080 - accuracy: 0.9073 - val_loss: 0.4286 - val_accuracy: 0.8308
Epoch 773/1000
453/453 [==============================] - 0s 100us/step - loss: 0.2100 - accuracy: 0.9073 - val_loss: 0.4225 - val_accuracy: 0.8308
Epoch 774/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2097 - accuracy: 0.9073 - val_loss: 0.4167 - val_accuracy: 0.8308
Epoch 775/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2077 - accuracy: 0.9073 - val_loss: 0.4093 - val_accuracy: 0.8308
Epoch 776/1000
453/453 [==============================] - 0s 131us/step - loss: 0.2070 - accuracy: 0.9073 - val_loss: 0.4068 - val_accuracy: 0.8308
Epoch 777/1000
453/453 [==============================] - 0s 150us/step - loss: 0.2083 - accuracy: 0.9073 - val_loss: 0.4061 - val_accuracy: 0.8308
Epoch 778/1000
453/453 [==============================] - 0s 163us/step - loss: 0.2065 - accuracy: 0.9073 - val_loss: 0.4084 - 

453/453 [==============================] - 0s 126us/step - loss: 0.2065 - accuracy: 0.9073 - val_loss: 0.4018 - val_accuracy: 0.8308
Epoch 883/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2072 - accuracy: 0.9073 - val_loss: 0.4020 - val_accuracy: 0.8308
Epoch 884/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2083 - accuracy: 0.9073 - val_loss: 0.4029 - val_accuracy: 0.8308
Epoch 885/1000
453/453 [==============================] - 0s 139us/step - loss: 0.2073 - accuracy: 0.9073 - val_loss: 0.4062 - val_accuracy: 0.8308
Epoch 886/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2080 - accuracy: 0.9073 - val_loss: 0.4030 - val_accuracy: 0.8308
Epoch 887/1000
453/453 [==============================] - 0s 139us/step - loss: 0.2086 - accuracy: 0.9073 - val_loss: 0.3989 - val_accuracy: 0.8308
Epoch 888/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2103 - accuracy: 0.9073 - val_loss: 0.4030 - 

453/453 [==============================] - 0s 149us/step - loss: 0.2082 - accuracy: 0.9073 - val_loss: 0.3973 - val_accuracy: 0.8308
Epoch 993/1000
453/453 [==============================] - 0s 132us/step - loss: 0.2072 - accuracy: 0.9073 - val_loss: 0.3998 - val_accuracy: 0.8308
Epoch 994/1000
453/453 [==============================] - 0s 103us/step - loss: 0.2087 - accuracy: 0.9073 - val_loss: 0.4011 - val_accuracy: 0.8308
Epoch 995/1000
453/453 [==============================] - 0s 102us/step - loss: 0.2087 - accuracy: 0.9073 - val_loss: 0.4048 - val_accuracy: 0.8308
Epoch 996/1000
453/453 [==============================] - 0s 144us/step - loss: 0.2071 - accuracy: 0.9073 - val_loss: 0.3999 - val_accuracy: 0.8308
Epoch 997/1000
453/453 [==============================] - 0s 322us/step - loss: 0.2132 - accuracy: 0.9029 - val_loss: 0.4023 - val_accuracy: 0.8308
Epoch 998/1000
453/453 [==============================] - 0s 341us/step - loss: 0.2084 - accuracy: 0.9073 - val_loss: 0.3975 - 

In [38]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 90.71%


In [21]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [22]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [23]:
y_prob = df_proba[df_proba['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [1.7294256e-01, 8.2705740e-01, 5.5157623e-12],
       [2.5316253e-01, 7.4683750e-01, 1.1009565e-10],
       [5.7847735e-09, 9.2700670e-09, 1.0000000e+00],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [0.0000000e+00, 1.0000000e+00, 7.5481790e-20],
       [1.0000000e+00, 3.4427258e-22, 1.2000968e-10],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [3.1088975e-18, 1.0000000e+00, 9.4137320e-13],
       [1.9179791e-22, 1.0000000e+00, 7.7481400e-34],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [5.7847735e-09, 9.2700670e-09, 1.0000000e+00],
       [7.3523060e-01, 2.6476938e-01, 2.7420820e-10],
       [2.9149520e-01, 7.085

In [24]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [25]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.933491124260355

In [26]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [27]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.933491124260355

In [28]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [29]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [30]:
dat2

,0,test
0,NRS110,2
1,NRS254,1
2,BCH-SA-09,0
3,NRS177,0
4,GA27,1
...,...,...
190,NRS001,1
191,NRS209,2
192,NRS272,0
193,NRS110,2


In [31]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 326us/step - loss: 0.9344 - accuracy: 0.5298 - val_loss: 0.7770 - val_accuracy: 0.7795
Epoch 2/1000
453/453 [==============================] - 0s 135us/step - loss: 0.6988 - accuracy: 0.7859 - val_loss: 0.6353 - val_accuracy: 0.8256
Epoch 3/1000
453/453 [==============================] - 0s 246us/step - loss: 0.5811 - accuracy: 0.7969 - val_loss: 0.5465 - val_accuracy: 0.7949
Epoch 4/1000
453/453 [==============================] - 0s 181us/step - loss: 0.5121 - accuracy: 0.8013 - val_loss: 0.4804 - val_accuracy: 0.8256
Epoch 5/1000
453/453 [==============================] - 0s 189us/step - loss: 0.4660 - accuracy: 0.7837 - val_loss: 0.4428 - val_accuracy: 0.8154
Epoch 6/1000
453/453 [==============================] - 0s 188us/step - loss: 0.4387 - accuracy: 0.8013 - val_loss: 0.4212 - val_accuracy: 0.8000
Epoch 7/1000
453/453 [==============================] - 0s 200us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 98us/step - loss: 0.2731 - accuracy: 0.8720 - val_loss: 0.3067 - val_accuracy: 0.8513
Epoch 113/1000
453/453 [==============================] - 0s 157us/step - loss: 0.2635 - accuracy: 0.8587 - val_loss: 0.3065 - val_accuracy: 0.8564
Epoch 114/1000
453/453 [==============================] - 0s 202us/step - loss: 0.2655 - accuracy: 0.8786 - val_loss: 0.3044 - val_accuracy: 0.8462
Epoch 115/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2593 - accuracy: 0.8698 - val_loss: 0.3032 - val_accuracy: 0.8564
Epoch 116/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2567 - accuracy: 0.8764 - val_loss: 0.3088 - val_accuracy: 0.8513
Epoch 117/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2669 - accuracy: 0.8830 - val_loss: 0.3131 - val_accuracy: 0.8513
Epoch 118/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2609 - accuracy: 0.8675 - val_l

453/453 [==============================] - 0s 137us/step - loss: 0.2433 - accuracy: 0.8764 - val_loss: 0.3271 - val_accuracy: 0.8462
Epoch 223/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2438 - accuracy: 0.8852 - val_loss: 0.3346 - val_accuracy: 0.8564
Epoch 224/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2399 - accuracy: 0.8852 - val_loss: 0.3368 - val_accuracy: 0.8564
Epoch 225/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2395 - accuracy: 0.8830 - val_loss: 0.3308 - val_accuracy: 0.8564
Epoch 226/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2398 - accuracy: 0.8874 - val_loss: 0.3334 - val_accuracy: 0.8564
Epoch 227/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2428 - accuracy: 0.8808 - val_loss: 0.3402 - val_accuracy: 0.8410
Epoch 228/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2434 - accuracy: 0.8808 - val_loss: 0.3251 - 

453/453 [==============================] - 0s 114us/step - loss: 0.2398 - accuracy: 0.8852 - val_loss: 0.3417 - val_accuracy: 0.8462
Epoch 333/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2453 - accuracy: 0.8830 - val_loss: 0.3514 - val_accuracy: 0.8462
Epoch 334/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2355 - accuracy: 0.8786 - val_loss: 0.3426 - val_accuracy: 0.8462
Epoch 335/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2382 - accuracy: 0.8830 - val_loss: 0.3444 - val_accuracy: 0.8462
Epoch 336/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2356 - accuracy: 0.8852 - val_loss: 0.3443 - val_accuracy: 0.8462
Epoch 337/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2352 - accuracy: 0.8852 - val_loss: 0.3459 - val_accuracy: 0.8359
Epoch 338/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2346 - accuracy: 0.8852 - val_loss: 0.3433 - 

453/453 [==============================] - 0s 110us/step - loss: 0.2356 - accuracy: 0.8808 - val_loss: 0.3639 - val_accuracy: 0.8462
Epoch 443/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2318 - accuracy: 0.8786 - val_loss: 0.3637 - val_accuracy: 0.8462
Epoch 444/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2337 - accuracy: 0.8874 - val_loss: 0.3697 - val_accuracy: 0.8462
Epoch 445/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2356 - accuracy: 0.8830 - val_loss: 0.3688 - val_accuracy: 0.8462
Epoch 446/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2332 - accuracy: 0.8830 - val_loss: 0.3652 - val_accuracy: 0.8462
Epoch 447/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2319 - accuracy: 0.8918 - val_loss: 0.3678 - val_accuracy: 0.8513
Epoch 448/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2455 - accuracy: 0.8786 - val_loss: 0.3710 - 

453/453 [==============================] - 0s 87us/step - loss: 0.2311 - accuracy: 0.8874 - val_loss: 0.3825 - val_accuracy: 0.8513
Epoch 553/1000
453/453 [==============================] - 0s 83us/step - loss: 0.2317 - accuracy: 0.8808 - val_loss: 0.3829 - val_accuracy: 0.8513
Epoch 554/1000
453/453 [==============================] - 0s 156us/step - loss: 0.2345 - accuracy: 0.8874 - val_loss: 0.3859 - val_accuracy: 0.8513
Epoch 555/1000
453/453 [==============================] - 0s 204us/step - loss: 0.2314 - accuracy: 0.8874 - val_loss: 0.3835 - val_accuracy: 0.8513
Epoch 556/1000
453/453 [==============================] - 0s 145us/step - loss: 0.2307 - accuracy: 0.8830 - val_loss: 0.3842 - val_accuracy: 0.8513
Epoch 557/1000
453/453 [==============================] - 0s 143us/step - loss: 0.2339 - accuracy: 0.8808 - val_loss: 0.3886 - val_accuracy: 0.8410
Epoch 558/1000
453/453 [==============================] - 0s 164us/step - loss: 0.2307 - accuracy: 0.8874 - val_loss: 0.3844 - va

453/453 [==============================] - 0s 107us/step - loss: 0.2330 - accuracy: 0.8874 - val_loss: 0.4162 - val_accuracy: 0.8462
Epoch 663/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2311 - accuracy: 0.8874 - val_loss: 0.4138 - val_accuracy: 0.8462
Epoch 664/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2316 - accuracy: 0.8874 - val_loss: 0.4168 - val_accuracy: 0.8462
Epoch 665/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2318 - accuracy: 0.8764 - val_loss: 0.4117 - val_accuracy: 0.8513
Epoch 666/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2321 - accuracy: 0.8874 - val_loss: 0.4135 - val_accuracy: 0.8513
Epoch 667/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2332 - accuracy: 0.8786 - val_loss: 0.4144 - val_accuracy: 0.8462
Epoch 668/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2311 - accuracy: 0.8808 - val_loss: 0.4133 - 

453/453 [==============================] - 0s 116us/step - loss: 0.2294 - accuracy: 0.8874 - val_loss: 0.4424 - val_accuracy: 0.8462
Epoch 773/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2307 - accuracy: 0.8874 - val_loss: 0.4458 - val_accuracy: 0.8462
Epoch 774/1000
453/453 [==============================] - 0s 153us/step - loss: 0.2316 - accuracy: 0.8830 - val_loss: 0.4514 - val_accuracy: 0.8308
Epoch 775/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2299 - accuracy: 0.8830 - val_loss: 0.4447 - val_accuracy: 0.8462
Epoch 776/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2313 - accuracy: 0.8830 - val_loss: 0.4445 - val_accuracy: 0.8462
Epoch 777/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2323 - accuracy: 0.8874 - val_loss: 0.4431 - val_accuracy: 0.8462
Epoch 778/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2300 - accuracy: 0.8874 - val_loss: 0.4439 - 

453/453 [==============================] - 0s 109us/step - loss: 0.2363 - accuracy: 0.8852 - val_loss: 0.4577 - val_accuracy: 0.8564
Epoch 883/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2331 - accuracy: 0.8808 - val_loss: 0.4581 - val_accuracy: 0.8564
Epoch 884/1000
453/453 [==============================] - 0s 103us/step - loss: 0.2336 - accuracy: 0.8874 - val_loss: 0.4596 - val_accuracy: 0.8564
Epoch 885/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2332 - accuracy: 0.8808 - val_loss: 0.4583 - val_accuracy: 0.8564
Epoch 886/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2309 - accuracy: 0.8874 - val_loss: 0.4597 - val_accuracy: 0.8564
Epoch 887/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2320 - accuracy: 0.8874 - val_loss: 0.4596 - val_accuracy: 0.8564
Epoch 888/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2324 - accuracy: 0.8852 - val_loss: 0.4674 - 

453/453 [==============================] - 0s 107us/step - loss: 0.2311 - accuracy: 0.8874 - val_loss: 0.4860 - val_accuracy: 0.8513
Epoch 993/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2294 - accuracy: 0.8874 - val_loss: 0.4916 - val_accuracy: 0.8513
Epoch 994/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2326 - accuracy: 0.8852 - val_loss: 0.4898 - val_accuracy: 0.8564
Epoch 995/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2286 - accuracy: 0.8874 - val_loss: 0.4860 - val_accuracy: 0.8564
Epoch 996/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2322 - accuracy: 0.8830 - val_loss: 0.4890 - val_accuracy: 0.8513
Epoch 997/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2293 - accuracy: 0.8874 - val_loss: 0.4884 - val_accuracy: 0.8564
Epoch 998/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2314 - accuracy: 0.8786 - val_loss: 0.4897 - 

In [46]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

195/195 [==============================] - 0s 56us/step
over-sampling test accuracy: 84.62%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 1, 0, 1, 1, 0, 1, 1, 0, 2, 0, 0, 2, 2, 1, 2, 0, 0, 1, 0, 0, 0,
       2, 1, 0, 0, 0, 1, 0, 2, 0, 2, 1, 2, 1, 1, 2, 0, 1, 2, 1, 2, 2, 0,
       2, 0, 0, 2, 2, 0, 1, 0, 1, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2,
       1, 0, 2, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 0, 2, 2, 0, 2, 0, 0, 1, 2, 1, 1,
       2, 2, 1, 2, 2, 1, 0, 0, 2, 0, 0, 0, 2, 2, 1, 2, 0, 2, 2, 0, 0, 2,
       0, 0, 1, 2, 2, 0, 1, 1, 2, 1, 0, 0, 2, 1, 2, 1, 0, 2, 1, 2, 1, 1,
       1, 2, 1, 2, 0, 1, 0, 1, 2, 1, 2, 1, 2, 0, 0, 2, 1, 2, 2, 1, 0, 2,
       1, 2, 0, 2, 0, 1, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS110,2,2
1,NRS254,1,1
2,BCH-SA-09,0,0
3,NRS177,0,1
4,GA27,1,1
...,...,...,...
190,NRS001,1,0
191,NRS209,2,2
192,NRS272,0,0
193,NRS110,2,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,2.240305e-11,3.785083e-08,1.000000e+00
1,1.868346e-30,1.000000e+00,1.587276e-25
2,8.095264e-01,1.904736e-01,5.845330e-09
3,2.929480e-02,9.707052e-01,1.982460e-08
4,2.331409e-01,7.668591e-01,1.356124e-07
...,...,...,...
190,5.476567e-01,4.523419e-01,1.395371e-06
191,1.173925e-08,7.917819e-09,1.000000e+00
192,9.999931e-01,9.342754e-07,5.903541e-06
193,2.240305e-11,3.785083e-08,1.000000e+00


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p17s.csv", index = False,
         header=None)

In [50]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 182us/step - loss: 0.2305 - accuracy: 0.8874 - val_loss: 0.4641 - val_accuracy: 0.8462
Epoch 2/1000
453/453 [==============================] - 0s 151us/step - loss: 0.2288 - accuracy: 0.8852 - val_loss: 0.4648 - val_accuracy: 0.8462
Epoch 3/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2337 - accuracy: 0.8896 - val_loss: 0.4610 - val_accuracy: 0.8410
Epoch 4/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2421 - accuracy: 0.8786 - val_loss: 0.4594 - val_accuracy: 0.8462
Epoch 5/1000
453/453 [==============================] - 0s 99us/step - loss: 0.2288 - accuracy: 0.8874 - val_loss: 0.4639 - val_accuracy: 0.8462
Epoch 6/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2304 - accuracy: 0.8852 - val_loss: 0.4649 - val_accuracy: 0.8462
Epoch 7/1000
453/453 [==============================] - 0s 100us/step - loss: 0

Epoch 112/1000
453/453 [==============================] - 0s 165us/step - loss: 0.2294 - accuracy: 0.8830 - val_loss: 0.4835 - val_accuracy: 0.8359
Epoch 113/1000
453/453 [==============================] - 0s 261us/step - loss: 0.2294 - accuracy: 0.8808 - val_loss: 0.4770 - val_accuracy: 0.8564
Epoch 114/1000
453/453 [==============================] - 0s 270us/step - loss: 0.2293 - accuracy: 0.8896 - val_loss: 0.4822 - val_accuracy: 0.8410
Epoch 115/1000
453/453 [==============================] - 0s 195us/step - loss: 0.2295 - accuracy: 0.8830 - val_loss: 0.4833 - val_accuracy: 0.8410
Epoch 116/1000
453/453 [==============================] - 0s 170us/step - loss: 0.2310 - accuracy: 0.8874 - val_loss: 0.4861 - val_accuracy: 0.8410
Epoch 117/1000
453/453 [==============================] - 0s 133us/step - loss: 0.2313 - accuracy: 0.8874 - val_loss: 0.4848 - val_accuracy: 0.8410
Epoch 118/1000
453/453 [==============================] - 0s 207us/step - loss: 0.2281 - accuracy: 0.8874 - val_

453/453 [==============================] - 0s 110us/step - loss: 0.2298 - accuracy: 0.8874 - val_loss: 0.4988 - val_accuracy: 0.8462
Epoch 223/1000
453/453 [==============================] - 0s 102us/step - loss: 0.2284 - accuracy: 0.8830 - val_loss: 0.4949 - val_accuracy: 0.8462
Epoch 224/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2283 - accuracy: 0.8852 - val_loss: 0.4974 - val_accuracy: 0.8462
Epoch 225/1000
453/453 [==============================] - 0s 102us/step - loss: 0.2273 - accuracy: 0.8874 - val_loss: 0.4958 - val_accuracy: 0.8462
Epoch 226/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2286 - accuracy: 0.8874 - val_loss: 0.4948 - val_accuracy: 0.8462
Epoch 227/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2305 - accuracy: 0.8874 - val_loss: 0.4950 - val_accuracy: 0.8462
Epoch 228/1000
453/453 [==============================] - 0s 103us/step - loss: 0.2382 - accuracy: 0.8874 - val_loss: 0.5051 - 

453/453 [==============================] - 0s 200us/step - loss: 0.2329 - accuracy: 0.8830 - val_loss: 0.5085 - val_accuracy: 0.8359
Epoch 333/1000
453/453 [==============================] - 0s 159us/step - loss: 0.2265 - accuracy: 0.8874 - val_loss: 0.5123 - val_accuracy: 0.8462
Epoch 334/1000
453/453 [==============================] - 0s 137us/step - loss: 0.2303 - accuracy: 0.8874 - val_loss: 0.5119 - val_accuracy: 0.8462
Epoch 335/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2280 - accuracy: 0.8830 - val_loss: 0.5083 - val_accuracy: 0.8462
Epoch 336/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2309 - accuracy: 0.8808 - val_loss: 0.5079 - val_accuracy: 0.8462
Epoch 337/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2301 - accuracy: 0.8874 - val_loss: 0.5139 - val_accuracy: 0.8462
Epoch 338/1000
453/453 [==============================] - 0s 137us/step - loss: 0.2285 - accuracy: 0.8874 - val_loss: 0.5138 - 

453/453 [==============================] - 0s 191us/step - loss: 0.2293 - accuracy: 0.8874 - val_loss: 0.5092 - val_accuracy: 0.8564
Epoch 443/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2281 - accuracy: 0.8874 - val_loss: 0.5061 - val_accuracy: 0.8564
Epoch 444/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2277 - accuracy: 0.8874 - val_loss: 0.5076 - val_accuracy: 0.8564
Epoch 445/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2289 - accuracy: 0.8874 - val_loss: 0.5073 - val_accuracy: 0.8564
Epoch 446/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2293 - accuracy: 0.8874 - val_loss: 0.5100 - val_accuracy: 0.8564
Epoch 447/1000
453/453 [==============================] - 0s 102us/step - loss: 0.2273 - accuracy: 0.8874 - val_loss: 0.5072 - val_accuracy: 0.8564
Epoch 448/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2278 - accuracy: 0.8874 - val_loss: 0.5087 - 

453/453 [==============================] - 0s 178us/step - loss: 0.2271 - accuracy: 0.8874 - val_loss: 0.5098 - val_accuracy: 0.8564
Epoch 553/1000
453/453 [==============================] - 0s 142us/step - loss: 0.2297 - accuracy: 0.8874 - val_loss: 0.5067 - val_accuracy: 0.8564
Epoch 554/1000
453/453 [==============================] - 0s 452us/step - loss: 0.2279 - accuracy: 0.8874 - val_loss: 0.5085 - val_accuracy: 0.8564
Epoch 555/1000
453/453 [==============================] - 0s 222us/step - loss: 0.2284 - accuracy: 0.8874 - val_loss: 0.5081 - val_accuracy: 0.8564
Epoch 556/1000
453/453 [==============================] - 0s 169us/step - loss: 0.2270 - accuracy: 0.8874 - val_loss: 0.5088 - val_accuracy: 0.8564
Epoch 557/1000
453/453 [==============================] - 0s 142us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 0.5102 - val_accuracy: 0.8564
Epoch 558/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2307 - accuracy: 0.8830 - val_loss: 0.5111 - 

Epoch 663/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2307 - accuracy: 0.8852 - val_loss: 0.5282 - val_accuracy: 0.8564
Epoch 664/1000
453/453 [==============================] - 0s 139us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 0.5317 - val_accuracy: 0.8564
Epoch 665/1000
453/453 [==============================] - 0s 134us/step - loss: 0.2264 - accuracy: 0.8874 - val_loss: 0.5284 - val_accuracy: 0.8564
Epoch 666/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2279 - accuracy: 0.8874 - val_loss: 0.5270 - val_accuracy: 0.8564
Epoch 667/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2286 - accuracy: 0.8874 - val_loss: 0.5302 - val_accuracy: 0.8564
Epoch 668/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2294 - accuracy: 0.8874 - val_loss: 0.5327 - val_accuracy: 0.8564
Epoch 669/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2297 - accuracy: 0.8874 - val_

453/453 [==============================] - 0s 112us/step - loss: 0.2278 - accuracy: 0.8830 - val_loss: 0.5295 - val_accuracy: 0.8564
Epoch 774/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2276 - accuracy: 0.8874 - val_loss: 0.5323 - val_accuracy: 0.8564
Epoch 775/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2295 - accuracy: 0.8874 - val_loss: 0.5301 - val_accuracy: 0.8564
Epoch 776/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2312 - accuracy: 0.8830 - val_loss: 0.5357 - val_accuracy: 0.8564
Epoch 777/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2330 - accuracy: 0.8874 - val_loss: 0.5350 - val_accuracy: 0.8564
Epoch 778/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2296 - accuracy: 0.8874 - val_loss: 0.5358 - val_accuracy: 0.8564
Epoch 779/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2281 - accuracy: 0.8874 - val_loss: 0.5300 - 

453/453 [==============================] - 0s 106us/step - loss: 0.2335 - accuracy: 0.8852 - val_loss: 0.5566 - val_accuracy: 0.8564
Epoch 884/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2308 - accuracy: 0.8830 - val_loss: 0.5564 - val_accuracy: 0.8564
Epoch 885/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2295 - accuracy: 0.8874 - val_loss: 0.5536 - val_accuracy: 0.8564
Epoch 886/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2280 - accuracy: 0.8874 - val_loss: 0.5599 - val_accuracy: 0.8564
Epoch 887/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2283 - accuracy: 0.8874 - val_loss: 0.5571 - val_accuracy: 0.8564
Epoch 888/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2275 - accuracy: 0.8874 - val_loss: 0.5579 - val_accuracy: 0.8564
Epoch 889/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 0.5596 - 

Epoch 938/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 0.5634 - val_accuracy: 0.8564
Epoch 939/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2267 - accuracy: 0.8874 - val_loss: 0.5649 - val_accuracy: 0.8564
Epoch 940/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2290 - accuracy: 0.8808 - val_loss: 0.5646 - val_accuracy: 0.8564
Epoch 941/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2273 - accuracy: 0.8874 - val_loss: 0.5633 - val_accuracy: 0.8564
Epoch 942/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2279 - accuracy: 0.8874 - val_loss: 0.5654 - val_accuracy: 0.8564
Epoch 943/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2305 - accuracy: 0.8830 - val_loss: 0.5625 - val_accuracy: 0.8462
Epoch 944/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2262 - accuracy: 0.8874 - val_

In [51]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 88.58%


In [32]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [33]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [34]:
y_prob2 = df_proba2[df_proba2['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[2.2403053e-11, 3.7850830e-08, 1.0000000e+00],
       [1.8683458e-30, 1.0000000e+00, 1.5872764e-25],
       [8.0952640e-01, 1.9047360e-01, 5.8453300e-09],
       [2.9294800e-02, 9.7070515e-01, 1.9824599e-08],
       [2.3314092e-01, 7.6685905e-01, 1.3561241e-07],
       [6.9079690e-01, 3.0920310e-01, 2.4776810e-09],
       [2.6571120e-04, 9.9973420e-01, 2.3860011e-11],
       [3.5755882e-05, 9.9996424e-01, 4.0667150e-09],
       [9.9998700e-01, 1.3037817e-05, 1.4557464e-12],
       [2.2403053e-11, 3.7850830e-08, 1.0000000e+00],
       [5.4765666e-01, 4.5234194e-01, 1.3953713e-06],
       [6.9079690e-01, 3.0920310e-01, 2.4776810e-09],
       [1.1739245e-08, 7.9178190e-09, 1.0000000e+00],
       [1.1739245e-08, 7.9178190e-09, 1.0000000e+00],
       [3.5508074e-35, 1.0000000e+00, 1.9292610e-26],
       [2.2403053e-11, 3.7850830e-08, 1.0000000e+00],
       [6.2456760e-01, 3.7543230e-01, 2.2127670e-09],
       [5.4765666e-01, 4.5234194e-01, 1.3953713e-06],
       [4.2269326e-07, 9.999

In [35]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9491518737672585

In [36]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9491518737672585

In [37]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [38]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [39]:
dat3

,0,test
0,NRS249,1
1,NRS172,1
2,NRS209,2
3,NRS108,1
4,NRS209,2
...,...,...
190,NRS209,2
191,NRS110,2
192,NRS255,1
193,NRS175,1


In [40]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [41]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 332us/step - loss: 0.8319 - accuracy: 0.6380 - val_loss: 0.6758 - val_accuracy: 0.6513
Epoch 2/1000
453/453 [==============================] - 0s 220us/step - loss: 0.6140 - accuracy: 0.7064 - val_loss: 0.5367 - val_accuracy: 0.7231
Epoch 3/1000
453/453 [==============================] - 0s 197us/step - loss: 0.5266 - accuracy: 0.7285 - val_loss: 0.4662 - val_accuracy: 0.7538
Epoch 4/1000
453/453 [==============================] - 0s 145us/step - loss: 0.4733 - accuracy: 0.7616 - val_loss: 0.4347 - val_accuracy: 0.7385
Epoch 5/1000
453/453 [==============================] - 0s 139us/step - loss: 0.4429 - accuracy: 0.7770 - val_loss: 0.4002 - val_accuracy: 0.7846
Epoch 6/1000
453/453 [==============================] - 0s 192us/step - loss: 0.4258 - accuracy: 0.8079 - val_loss: 0.3868 - val_accuracy: 0.8051
Epoch 7/1000
453/453 [==============================] - 0s 231us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 190us/step - loss: 0.2512 - accuracy: 0.8874 - val_loss: 0.2755 - val_accuracy: 0.8667
Epoch 113/1000
453/453 [==============================] - 0s 189us/step - loss: 0.2628 - accuracy: 0.8786 - val_loss: 0.2800 - val_accuracy: 0.8667
Epoch 114/1000
453/453 [==============================] - 0s 197us/step - loss: 0.2568 - accuracy: 0.8830 - val_loss: 0.2685 - val_accuracy: 0.8718
Epoch 115/1000
453/453 [==============================] - 0s 201us/step - loss: 0.2529 - accuracy: 0.8808 - val_loss: 0.2686 - val_accuracy: 0.8615
Epoch 116/1000
453/453 [==============================] - 0s 181us/step - loss: 0.2564 - accuracy: 0.8830 - val_loss: 0.2704 - val_accuracy: 0.8667
Epoch 117/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2586 - accuracy: 0.8808 - val_loss: 0.2695 - val_accuracy: 0.8667
Epoch 118/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2589 - accuracy: 0.8896 - val_

453/453 [==============================] - 0s 168us/step - loss: 0.2362 - accuracy: 0.8874 - val_loss: 0.2680 - val_accuracy: 0.8667
Epoch 223/1000
453/453 [==============================] - 0s 175us/step - loss: 0.2417 - accuracy: 0.8808 - val_loss: 0.2868 - val_accuracy: 0.8667
Epoch 224/1000
453/453 [==============================] - 0s 181us/step - loss: 0.2347 - accuracy: 0.8852 - val_loss: 0.2705 - val_accuracy: 0.8667
Epoch 225/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2363 - accuracy: 0.8896 - val_loss: 0.2665 - val_accuracy: 0.8564
Epoch 226/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2318 - accuracy: 0.8830 - val_loss: 0.3019 - val_accuracy: 0.8564
Epoch 227/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2358 - accuracy: 0.8874 - val_loss: 0.2838 - val_accuracy: 0.8564
Epoch 228/1000
453/453 [==============================] - 0s 174us/step - loss: 0.2332 - accuracy: 0.8874 - val_loss: 0.2683 - 

453/453 [==============================] - 0s 183us/step - loss: 0.2257 - accuracy: 0.8874 - val_loss: 0.2762 - val_accuracy: 0.8564
Epoch 333/1000
453/453 [==============================] - 0s 184us/step - loss: 0.2278 - accuracy: 0.8852 - val_loss: 0.2711 - val_accuracy: 0.8667
Epoch 334/1000
453/453 [==============================] - 0s 193us/step - loss: 0.2367 - accuracy: 0.8852 - val_loss: 0.3075 - val_accuracy: 0.8564
Epoch 335/1000
453/453 [==============================] - 0s 182us/step - loss: 0.2290 - accuracy: 0.8918 - val_loss: 0.2730 - val_accuracy: 0.8564
Epoch 336/1000
453/453 [==============================] - 0s 187us/step - loss: 0.2269 - accuracy: 0.8918 - val_loss: 0.2675 - val_accuracy: 0.8667
Epoch 337/1000
453/453 [==============================] - 0s 161us/step - loss: 0.2258 - accuracy: 0.8918 - val_loss: 0.2897 - val_accuracy: 0.8564
Epoch 338/1000
453/453 [==============================] - 0s 148us/step - loss: 0.2292 - accuracy: 0.8918 - val_loss: 0.2720 - 

453/453 [==============================] - 0s 115us/step - loss: 0.2262 - accuracy: 0.8874 - val_loss: 0.2649 - val_accuracy: 0.8872
Epoch 443/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2227 - accuracy: 0.8896 - val_loss: 0.2866 - val_accuracy: 0.8564
Epoch 444/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2207 - accuracy: 0.8918 - val_loss: 0.2726 - val_accuracy: 0.8718
Epoch 445/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2255 - accuracy: 0.8918 - val_loss: 0.2820 - val_accuracy: 0.8564
Epoch 446/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2231 - accuracy: 0.8918 - val_loss: 0.2929 - val_accuracy: 0.8564
Epoch 447/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2250 - accuracy: 0.8918 - val_loss: 0.2907 - val_accuracy: 0.8564
Epoch 448/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2258 - accuracy: 0.8852 - val_loss: 0.2826 - 

453/453 [==============================] - 0s 113us/step - loss: 0.2284 - accuracy: 0.8918 - val_loss: 0.3011 - val_accuracy: 0.8564
Epoch 553/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2223 - accuracy: 0.8896 - val_loss: 0.2851 - val_accuracy: 0.8615
Epoch 554/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2246 - accuracy: 0.8918 - val_loss: 0.2760 - val_accuracy: 0.8718
Epoch 555/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2252 - accuracy: 0.8940 - val_loss: 0.2867 - val_accuracy: 0.8564
Epoch 556/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2244 - accuracy: 0.8918 - val_loss: 0.2804 - val_accuracy: 0.8615
Epoch 557/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2229 - accuracy: 0.8896 - val_loss: 0.2808 - val_accuracy: 0.8615
Epoch 558/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2281 - accuracy: 0.8918 - val_loss: 0.2765 - 

453/453 [==============================] - 0s 115us/step - loss: 0.2230 - accuracy: 0.8918 - val_loss: 0.3019 - val_accuracy: 0.8564
Epoch 663/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2253 - accuracy: 0.8918 - val_loss: 0.3032 - val_accuracy: 0.8564
Epoch 664/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2230 - accuracy: 0.8918 - val_loss: 0.2883 - val_accuracy: 0.8615
Epoch 665/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2205 - accuracy: 0.8918 - val_loss: 0.2989 - val_accuracy: 0.8564
Epoch 666/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2240 - accuracy: 0.8918 - val_loss: 0.2967 - val_accuracy: 0.8564
Epoch 667/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2278 - accuracy: 0.8874 - val_loss: 0.3019 - val_accuracy: 0.8564
Epoch 668/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2251 - accuracy: 0.8918 - val_loss: 0.2853 - 

453/453 [==============================] - 0s 111us/step - loss: 0.2211 - accuracy: 0.8896 - val_loss: 0.3074 - val_accuracy: 0.8615
Epoch 773/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2279 - accuracy: 0.8918 - val_loss: 0.2955 - val_accuracy: 0.8615
Epoch 774/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2216 - accuracy: 0.8918 - val_loss: 0.2868 - val_accuracy: 0.8615
Epoch 775/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2194 - accuracy: 0.8918 - val_loss: 0.3009 - val_accuracy: 0.8615
Epoch 776/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2285 - accuracy: 0.8918 - val_loss: 0.3006 - val_accuracy: 0.8615
Epoch 777/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2199 - accuracy: 0.8896 - val_loss: 0.2936 - val_accuracy: 0.8615
Epoch 778/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2213 - accuracy: 0.8918 - val_loss: 0.3103 - 

453/453 [==============================] - 0s 137us/step - loss: 0.2275 - accuracy: 0.8918 - val_loss: 0.2968 - val_accuracy: 0.8564
Epoch 883/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2219 - accuracy: 0.8918 - val_loss: 0.2993 - val_accuracy: 0.8615
Epoch 884/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2272 - accuracy: 0.8918 - val_loss: 0.3129 - val_accuracy: 0.8564
Epoch 885/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2184 - accuracy: 0.8940 - val_loss: 0.2931 - val_accuracy: 0.8615
Epoch 886/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2186 - accuracy: 0.8940 - val_loss: 0.3031 - val_accuracy: 0.8564
Epoch 887/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2190 - accuracy: 0.8940 - val_loss: 0.3017 - val_accuracy: 0.8564
Epoch 888/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2189 - accuracy: 0.8918 - val_loss: 0.3022 - 

453/453 [==============================] - 0s 119us/step - loss: 0.2170 - accuracy: 0.8940 - val_loss: 0.2973 - val_accuracy: 0.8615
Epoch 993/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2207 - accuracy: 0.8940 - val_loss: 0.3021 - val_accuracy: 0.8615
Epoch 994/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2190 - accuracy: 0.8940 - val_loss: 0.3133 - val_accuracy: 0.8564
Epoch 995/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2176 - accuracy: 0.8940 - val_loss: 0.2941 - val_accuracy: 0.8615
Epoch 996/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2207 - accuracy: 0.8874 - val_loss: 0.3127 - val_accuracy: 0.8564
Epoch 997/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2205 - accuracy: 0.8940 - val_loss: 0.3029 - val_accuracy: 0.8564
Epoch 998/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2251 - accuracy: 0.8874 - val_loss: 0.2876 - 

In [59]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

195/195 [==============================] - 0s 59us/step
over-sampling test accuracy: 86.15%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([1, 1, 2, 1, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 2, 0, 2, 0, 0, 1, 2,
       1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 0, 1,
       2, 2, 2, 2, 2, 1, 2, 2, 0, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 1, 0, 2, 0, 2, 0, 1, 2,
       1, 2, 0, 0, 2, 0, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 0, 2, 0, 2, 0, 0,
       2, 0, 1, 1, 1, 1, 2, 2, 0, 1, 0, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0, 1,
       2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 1, 1, 1, 2, 0, 2, 2, 1, 2, 1, 0,
       1, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 1, 1, 2, 0, 1, 2, 2,
       0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 1, 0, 0, 2, 2, 2, 1, 0, 1])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS249,1,1
1,NRS172,1,1
2,NRS209,2,2
3,NRS108,1,1
4,NRS209,2,2
...,...,...,...
190,NRS209,2,2
191,NRS110,2,2
192,NRS255,1,1
193,NRS175,1,0


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,5.308234e-06,9.999946e-01,5.502656e-08
1,1.188718e-35,1.000000e+00,0.000000e+00
2,3.864103e-09,9.563090e-09,1.000000e+00
3,1.188718e-35,1.000000e+00,0.000000e+00
4,3.864103e-09,9.563090e-09,1.000000e+00
...,...,...,...
190,3.864103e-09,9.563090e-09,1.000000e+00
191,9.944022e-09,1.225267e-08,1.000000e+00
192,5.171805e-17,1.000000e+00,1.021424e-31
193,6.193884e-01,3.806116e-01,7.403208e-11


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p17s.csv", index = False,
         header=None)

In [63]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2205 - accuracy: 0.8940 - val_loss: 0.3294 - val_accuracy: 0.8667
Epoch 2/1000
453/453 [==============================] - 0s 135us/step - loss: 0.2183 - accuracy: 0.8918 - val_loss: 0.3476 - val_accuracy: 0.8615
Epoch 3/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2194 - accuracy: 0.8896 - val_loss: 0.3358 - val_accuracy: 0.8615
Epoch 4/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2164 - accuracy: 0.8940 - val_loss: 0.3447 - val_accuracy: 0.8615
Epoch 5/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2161 - accuracy: 0.8896 - val_loss: 0.3384 - val_accuracy: 0.8615
Epoch 6/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2192 - accuracy: 0.8918 - val_loss: 0.3434 - val_accuracy: 0.8615
Epoch 7/1000
453/453 [==============================] - 0s 105us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 182us/step - loss: 0.2185 - accuracy: 0.8940 - val_loss: 0.3514 - val_accuracy: 0.8615
Epoch 113/1000
453/453 [==============================] - 0s 169us/step - loss: 0.2178 - accuracy: 0.8874 - val_loss: 0.3469 - val_accuracy: 0.8615
Epoch 114/1000
453/453 [==============================] - 0s 145us/step - loss: 0.2203 - accuracy: 0.8940 - val_loss: 0.3581 - val_accuracy: 0.8615
Epoch 115/1000
453/453 [==============================] - 0s 191us/step - loss: 0.2259 - accuracy: 0.8830 - val_loss: 0.3456 - val_accuracy: 0.8615
Epoch 116/1000
453/453 [==============================] - 0s 188us/step - loss: 0.2187 - accuracy: 0.8940 - val_loss: 0.3311 - val_accuracy: 0.8615
Epoch 117/1000
453/453 [==============================] - 0s 274us/step - loss: 0.2161 - accuracy: 0.8918 - val_loss: 0.3397 - val_accuracy: 0.8615
Epoch 118/1000
453/453 [==============================] - 0s 370us/step - loss: 0.2262 - accuracy: 0.8918 - val_

453/453 [==============================] - 0s 123us/step - loss: 0.2197 - accuracy: 0.8874 - val_loss: 0.3377 - val_accuracy: 0.8615
Epoch 223/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2182 - accuracy: 0.8940 - val_loss: 0.3482 - val_accuracy: 0.8615
Epoch 224/1000
453/453 [==============================] - 0s 158us/step - loss: 0.2161 - accuracy: 0.8940 - val_loss: 0.3446 - val_accuracy: 0.8615
Epoch 225/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2203 - accuracy: 0.8940 - val_loss: 0.3574 - val_accuracy: 0.8615
Epoch 226/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2286 - accuracy: 0.8874 - val_loss: 0.3322 - val_accuracy: 0.8667
Epoch 227/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2272 - accuracy: 0.8874 - val_loss: 0.3487 - val_accuracy: 0.8615
Epoch 228/1000
453/453 [==============================] - 0s 132us/step - loss: 0.2262 - accuracy: 0.8940 - val_loss: 0.3538 - 

453/453 [==============================] - 0s 137us/step - loss: 0.2175 - accuracy: 0.8918 - val_loss: 0.3475 - val_accuracy: 0.8667
Epoch 333/1000
453/453 [==============================] - 0s 185us/step - loss: 0.2164 - accuracy: 0.8918 - val_loss: 0.3514 - val_accuracy: 0.8615
Epoch 334/1000
453/453 [==============================] - 0s 138us/step - loss: 0.2184 - accuracy: 0.8940 - val_loss: 0.3536 - val_accuracy: 0.8615
Epoch 335/1000
453/453 [==============================] - 0s 173us/step - loss: 0.2212 - accuracy: 0.8940 - val_loss: 0.3361 - val_accuracy: 0.8615
Epoch 336/1000
453/453 [==============================] - 0s 176us/step - loss: 0.2193 - accuracy: 0.8874 - val_loss: 0.3448 - val_accuracy: 0.8615
Epoch 337/1000
453/453 [==============================] - 0s 193us/step - loss: 0.2171 - accuracy: 0.8940 - val_loss: 0.3436 - val_accuracy: 0.8615
Epoch 338/1000
453/453 [==============================] - 0s 293us/step - loss: 0.2192 - accuracy: 0.8940 - val_loss: 0.3426 - 

453/453 [==============================] - 0s 92us/step - loss: 0.2158 - accuracy: 0.8896 - val_loss: 0.3494 - val_accuracy: 0.8615
Epoch 443/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2193 - accuracy: 0.8918 - val_loss: 0.3537 - val_accuracy: 0.8615
Epoch 444/1000
453/453 [==============================] - 0s 92us/step - loss: 0.2160 - accuracy: 0.8918 - val_loss: 0.3561 - val_accuracy: 0.8615
Epoch 445/1000
453/453 [==============================] - 0s 89us/step - loss: 0.2168 - accuracy: 0.8940 - val_loss: 0.3657 - val_accuracy: 0.8615
Epoch 446/1000
453/453 [==============================] - 0s 91us/step - loss: 0.2204 - accuracy: 0.8918 - val_loss: 0.3692 - val_accuracy: 0.8615
Epoch 447/1000
453/453 [==============================] - 0s 90us/step - loss: 0.2179 - accuracy: 0.8896 - val_loss: 0.3589 - val_accuracy: 0.8667
Epoch 448/1000
453/453 [==============================] - 0s 91us/step - loss: 0.2179 - accuracy: 0.8918 - val_loss: 0.3595 - val_ac

453/453 [==============================] - 0s 109us/step - loss: 0.2186 - accuracy: 0.8896 - val_loss: 0.3653 - val_accuracy: 0.8615
Epoch 553/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2154 - accuracy: 0.8918 - val_loss: 0.3607 - val_accuracy: 0.8667
Epoch 554/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2186 - accuracy: 0.8896 - val_loss: 0.3540 - val_accuracy: 0.8615
Epoch 555/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2196 - accuracy: 0.8940 - val_loss: 0.3462 - val_accuracy: 0.8667
Epoch 556/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2173 - accuracy: 0.8852 - val_loss: 0.3465 - val_accuracy: 0.8667
Epoch 557/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2161 - accuracy: 0.8918 - val_loss: 0.3533 - val_accuracy: 0.8615
Epoch 558/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2170 - accuracy: 0.8940 - val_loss: 0.3560 - 

453/453 [==============================] - 0s 110us/step - loss: 0.2183 - accuracy: 0.8896 - val_loss: 0.3327 - val_accuracy: 0.8667
Epoch 663/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2167 - accuracy: 0.8918 - val_loss: 0.3398 - val_accuracy: 0.8667
Epoch 664/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2167 - accuracy: 0.8940 - val_loss: 0.3411 - val_accuracy: 0.8615
Epoch 665/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2166 - accuracy: 0.8940 - val_loss: 0.3398 - val_accuracy: 0.8615
Epoch 666/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2154 - accuracy: 0.8940 - val_loss: 0.3364 - val_accuracy: 0.8615
Epoch 667/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2166 - accuracy: 0.8874 - val_loss: 0.3335 - val_accuracy: 0.8615
Epoch 668/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2185 - accuracy: 0.8940 - val_loss: 0.3422 - 

453/453 [==============================] - 0s 109us/step - loss: 0.2138 - accuracy: 0.8940 - val_loss: 0.3455 - val_accuracy: 0.8615
Epoch 773/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2156 - accuracy: 0.8940 - val_loss: 0.3532 - val_accuracy: 0.8615
Epoch 774/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2157 - accuracy: 0.8940 - val_loss: 0.3437 - val_accuracy: 0.8615
Epoch 775/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2153 - accuracy: 0.8940 - val_loss: 0.3419 - val_accuracy: 0.8615
Epoch 776/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2161 - accuracy: 0.8874 - val_loss: 0.3530 - val_accuracy: 0.8615
Epoch 777/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2168 - accuracy: 0.8940 - val_loss: 0.3436 - val_accuracy: 0.8615
Epoch 778/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2176 - accuracy: 0.8896 - val_loss: 0.3436 - 

453/453 [==============================] - 0s 107us/step - loss: 0.2147 - accuracy: 0.8940 - val_loss: 0.3537 - val_accuracy: 0.8615
Epoch 883/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2189 - accuracy: 0.8940 - val_loss: 0.3443 - val_accuracy: 0.8615
Epoch 884/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2209 - accuracy: 0.8918 - val_loss: 0.3546 - val_accuracy: 0.8615
Epoch 885/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2193 - accuracy: 0.8874 - val_loss: 0.3519 - val_accuracy: 0.8615
Epoch 886/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2154 - accuracy: 0.8918 - val_loss: 0.3495 - val_accuracy: 0.8615
Epoch 887/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2213 - accuracy: 0.8940 - val_loss: 0.3442 - val_accuracy: 0.8615
Epoch 888/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2159 - accuracy: 0.8874 - val_loss: 0.3455 - 

453/453 [==============================] - 0s 113us/step - loss: 0.2152 - accuracy: 0.8940 - val_loss: 0.3539 - val_accuracy: 0.8615
Epoch 993/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2179 - accuracy: 0.8896 - val_loss: 0.3572 - val_accuracy: 0.8615
Epoch 994/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2199 - accuracy: 0.8940 - val_loss: 0.3606 - val_accuracy: 0.8615
Epoch 995/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2150 - accuracy: 0.8940 - val_loss: 0.3534 - val_accuracy: 0.8615
Epoch 996/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2161 - accuracy: 0.8940 - val_loss: 0.3575 - val_accuracy: 0.8615
Epoch 997/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2168 - accuracy: 0.8962 - val_loss: 0.3543 - val_accuracy: 0.8667
Epoch 998/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2174 - accuracy: 0.8896 - val_loss: 0.3627 - 

In [64]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 89.20%


In [42]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [43]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [44]:
y_prob3 = df_proba3[df_proba3['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[5.30823400e-06, 9.99994640e-01, 5.50265580e-08],
       [1.18871840e-35, 1.00000000e+00, 0.00000000e+00],
       [3.86410330e-09, 9.56309000e-09, 1.00000000e+00],
       [1.18871840e-35, 1.00000000e+00, 0.00000000e+00],
       [3.86410330e-09, 9.56309000e-09, 1.00000000e+00],
       [6.78003200e-01, 3.21996870e-01, 4.50554180e-11],
       [6.78003200e-01, 3.21996870e-01, 4.50554180e-11],
       [5.30823400e-06, 9.99994640e-01, 5.50265580e-08],
       [8.37566200e-02, 9.16243430e-01, 2.59588990e-08],
       [1.18871840e-35, 1.00000000e+00, 0.00000000e+00],
       [9.99961730e-01, 3.82956680e-05, 3.64965120e-08],
       [6.78003200e-01, 3.21996870e-01, 4.50554180e-11],
       [1.18871840e-35, 1.00000000e+00, 0.00000000e+00],
       [5.17180520e-17, 1.00000000e+00, 1.02142410e-31],
       [6.19388400e-01, 3.80611630e-01, 7.40320800e-11],
       [9.94402200e-09, 1.22526735e-08, 1.00000000e+00],
       [7.25352900e-01, 2.74647000e-01, 7.35279140e-08],
       [9.94402200e-09, 1.22526

In [46]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9636686390532544

In [48]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9636686390532544

In [49]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [50]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [51]:
dat4

,0,test
0,NRS241,1
1,BCH-SA-01,1
2,NRS219,1
3,NRS209,2
4,NRS001,1
...,...,...
190,GA15,1
191,NRS246,1
192,115,1
193,312,1


In [52]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 448us/step - loss: 0.8501 - accuracy: 0.7020 - val_loss: 0.6750 - val_accuracy: 0.7179
Epoch 2/1000
453/453 [==============================] - 0s 253us/step - loss: 0.6133 - accuracy: 0.7594 - val_loss: 0.5578 - val_accuracy: 0.7128
Epoch 3/1000
453/453 [==============================] - 0s 212us/step - loss: 0.5148 - accuracy: 0.7815 - val_loss: 0.4737 - val_accuracy: 0.8359
Epoch 4/1000
453/453 [==============================] - 0s 205us/step - loss: 0.4762 - accuracy: 0.7947 - val_loss: 0.4469 - val_accuracy: 0.8410
Epoch 5/1000
453/453 [==============================] - 0s 184us/step - loss: 0.4600 - accuracy: 0.7704 - val_loss: 0.4376 - val_accuracy: 0.7282
Epoch 6/1000
453/453 [==============================] - 0s 186us/step - loss: 0.4216 - accuracy: 0.7903 - val_loss: 0.4192 - val_accuracy: 0.8205
Epoch 7/1000
453/453 [==============================] - 0s 201us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 253us/step - loss: 0.2698 - accuracy: 0.8786 - val_loss: 0.2850 - val_accuracy: 0.8769
Epoch 113/1000
453/453 [==============================] - 0s 272us/step - loss: 0.2652 - accuracy: 0.8786 - val_loss: 0.2902 - val_accuracy: 0.8769
Epoch 114/1000
453/453 [==============================] - 0s 185us/step - loss: 0.2783 - accuracy: 0.8675 - val_loss: 0.3117 - val_accuracy: 0.8769
Epoch 115/1000
453/453 [==============================] - 0s 221us/step - loss: 0.2734 - accuracy: 0.8653 - val_loss: 0.2823 - val_accuracy: 0.8769
Epoch 116/1000
453/453 [==============================] - 0s 232us/step - loss: 0.2625 - accuracy: 0.8764 - val_loss: 0.2780 - val_accuracy: 0.8769
Epoch 117/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2638 - accuracy: 0.8698 - val_loss: 0.2885 - val_accuracy: 0.8769
Epoch 118/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2711 - accuracy: 0.8742 - val_

453/453 [==============================] - 0s 119us/step - loss: 0.2491 - accuracy: 0.8786 - val_loss: 0.2776 - val_accuracy: 0.8769
Epoch 223/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2534 - accuracy: 0.8764 - val_loss: 0.2738 - val_accuracy: 0.8923
Epoch 224/1000
453/453 [==============================] - 0s 157us/step - loss: 0.2483 - accuracy: 0.8742 - val_loss: 0.2793 - val_accuracy: 0.8769
Epoch 225/1000
453/453 [==============================] - 0s 358us/step - loss: 0.2489 - accuracy: 0.8764 - val_loss: 0.2796 - val_accuracy: 0.8769
Epoch 226/1000
453/453 [==============================] - 0s 229us/step - loss: 0.2595 - accuracy: 0.8742 - val_loss: 0.2872 - val_accuracy: 0.8769
Epoch 227/1000
453/453 [==============================] - 0s 205us/step - loss: 0.2506 - accuracy: 0.8675 - val_loss: 0.2772 - val_accuracy: 0.8769
Epoch 228/1000
453/453 [==============================] - 0s 213us/step - loss: 0.2467 - accuracy: 0.8808 - val_loss: 0.2814 - 

453/453 [==============================] - 0s 115us/step - loss: 0.2465 - accuracy: 0.8786 - val_loss: 0.2824 - val_accuracy: 0.8923
Epoch 333/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2479 - accuracy: 0.8808 - val_loss: 0.2921 - val_accuracy: 0.8769
Epoch 334/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2463 - accuracy: 0.8852 - val_loss: 0.2849 - val_accuracy: 0.8769
Epoch 335/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2450 - accuracy: 0.8808 - val_loss: 0.2800 - val_accuracy: 0.8821
Epoch 336/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2433 - accuracy: 0.8764 - val_loss: 0.2826 - val_accuracy: 0.8718
Epoch 337/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2449 - accuracy: 0.8609 - val_loss: 0.2920 - val_accuracy: 0.8769
Epoch 338/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2427 - accuracy: 0.8764 - val_loss: 0.2821 - 

453/453 [==============================] - 0s 111us/step - loss: 0.2381 - accuracy: 0.8808 - val_loss: 0.2926 - val_accuracy: 0.8872
Epoch 443/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2391 - accuracy: 0.8830 - val_loss: 0.2914 - val_accuracy: 0.8872
Epoch 444/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2389 - accuracy: 0.8830 - val_loss: 0.2905 - val_accuracy: 0.8667
Epoch 445/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2395 - accuracy: 0.8698 - val_loss: 0.2886 - val_accuracy: 0.8718
Epoch 446/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2420 - accuracy: 0.8764 - val_loss: 0.2895 - val_accuracy: 0.8821
Epoch 447/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2455 - accuracy: 0.8764 - val_loss: 0.2982 - val_accuracy: 0.8718
Epoch 448/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2392 - accuracy: 0.8830 - val_loss: 0.3182 - 

453/453 [==============================] - 0s 113us/step - loss: 0.2339 - accuracy: 0.8830 - val_loss: 0.2935 - val_accuracy: 0.8718
Epoch 553/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2332 - accuracy: 0.8830 - val_loss: 0.2984 - val_accuracy: 0.8667
Epoch 554/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2392 - accuracy: 0.8764 - val_loss: 0.2955 - val_accuracy: 0.8821
Epoch 555/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2353 - accuracy: 0.8808 - val_loss: 0.2922 - val_accuracy: 0.8821
Epoch 556/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2377 - accuracy: 0.8786 - val_loss: 0.2941 - val_accuracy: 0.8718
Epoch 557/1000
453/453 [==============================] - 0s 128us/step - loss: 0.2397 - accuracy: 0.8720 - val_loss: 0.2917 - val_accuracy: 0.8718
Epoch 558/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2356 - accuracy: 0.8808 - val_loss: 0.2970 - 

453/453 [==============================] - 0s 129us/step - loss: 0.2293 - accuracy: 0.8874 - val_loss: 0.2954 - val_accuracy: 0.8769
Epoch 663/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2356 - accuracy: 0.8764 - val_loss: 0.2946 - val_accuracy: 0.8718
Epoch 664/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2300 - accuracy: 0.8675 - val_loss: 0.2978 - val_accuracy: 0.8667
Epoch 665/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2291 - accuracy: 0.8808 - val_loss: 0.2938 - val_accuracy: 0.8872
Epoch 666/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2298 - accuracy: 0.8720 - val_loss: 0.2940 - val_accuracy: 0.8821
Epoch 667/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2315 - accuracy: 0.8742 - val_loss: 0.2985 - val_accuracy: 0.8821
Epoch 668/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2299 - accuracy: 0.8764 - val_loss: 0.2950 - 

453/453 [==============================] - 0s 132us/step - loss: 0.2303 - accuracy: 0.8786 - val_loss: 0.3027 - val_accuracy: 0.8821
Epoch 773/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2280 - accuracy: 0.8808 - val_loss: 0.3040 - val_accuracy: 0.8872
Epoch 774/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2322 - accuracy: 0.8764 - val_loss: 0.3027 - val_accuracy: 0.8872
Epoch 775/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2290 - accuracy: 0.8742 - val_loss: 0.3064 - val_accuracy: 0.8821
Epoch 776/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2284 - accuracy: 0.8786 - val_loss: 0.3084 - val_accuracy: 0.8821
Epoch 777/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2286 - accuracy: 0.8852 - val_loss: 0.3111 - val_accuracy: 0.8821
Epoch 778/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2279 - accuracy: 0.8742 - val_loss: 0.3105 - 

453/453 [==============================] - 0s 173us/step - loss: 0.2306 - accuracy: 0.8808 - val_loss: 0.3162 - val_accuracy: 0.8769
Epoch 883/1000
453/453 [==============================] - 0s 177us/step - loss: 0.2271 - accuracy: 0.8786 - val_loss: 0.3138 - val_accuracy: 0.8923
Epoch 884/1000
453/453 [==============================] - 0s 146us/step - loss: 0.2283 - accuracy: 0.8808 - val_loss: 0.3140 - val_accuracy: 0.8769
Epoch 885/1000
453/453 [==============================] - 0s 146us/step - loss: 0.2262 - accuracy: 0.8852 - val_loss: 0.3195 - val_accuracy: 0.8667
Epoch 886/1000
453/453 [==============================] - 0s 148us/step - loss: 0.2271 - accuracy: 0.8786 - val_loss: 0.3196 - val_accuracy: 0.8718
Epoch 887/1000
453/453 [==============================] - 0s 147us/step - loss: 0.2315 - accuracy: 0.8808 - val_loss: 0.3145 - val_accuracy: 0.8821
Epoch 888/1000
453/453 [==============================] - 0s 149us/step - loss: 0.2264 - accuracy: 0.8786 - val_loss: 0.3144 - 

453/453 [==============================] - 0s 122us/step - loss: 0.2246 - accuracy: 0.8786 - val_loss: 0.3235 - val_accuracy: 0.8821
Epoch 993/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2356 - accuracy: 0.8830 - val_loss: 0.3243 - val_accuracy: 0.8769
Epoch 994/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2280 - accuracy: 0.8764 - val_loss: 0.3269 - val_accuracy: 0.8667
Epoch 995/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2278 - accuracy: 0.8830 - val_loss: 0.3227 - val_accuracy: 0.8821
Epoch 996/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2285 - accuracy: 0.8786 - val_loss: 0.3235 - val_accuracy: 0.8821
Epoch 997/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2259 - accuracy: 0.8830 - val_loss: 0.3207 - val_accuracy: 0.8821
Epoch 998/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2265 - accuracy: 0.8852 - val_loss: 0.3237 - 

In [72]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

195/195 [==============================] - 0s 56us/step
over-sampling test accuracy: 89.23%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 0, 2, 0, 2, 1, 1, 2, 2, 2, 0, 0, 1, 1, 2, 2, 2, 2, 0, 0, 1,
       0, 2, 0, 0, 0, 0, 2, 1, 2, 1, 2, 2, 2, 0, 0, 2, 1, 2, 1, 2, 1, 0,
       0, 0, 2, 2, 1, 0, 2, 0, 1, 2, 1, 2, 1, 0, 0, 0, 1, 1, 0, 1, 2, 0,
       1, 0, 0, 2, 1, 0, 2, 1, 1, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 1, 2, 2,
       2, 2, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0,
       2, 2, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 1,
       0, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 2, 1,
       1, 0, 0, 0, 0, 1, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 0, 0, 2, 0, 0,
       1, 2, 1, 0, 1, 2, 0, 2, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS241,1,1
1,BCH-SA-01,1,1
2,NRS219,1,0
3,NRS209,2,2
4,NRS001,1,0
...,...,...,...
190,GA15,1,0
191,NRS246,1,1
192,115,1,1
193,312,1,0


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,2.876360e-05,9.999713e-01,9.790981e-15
1,1.245487e-01,8.754513e-01,3.463491e-13
2,9.177501e-01,8.224987e-02,5.389061e-10
3,3.315624e-09,4.960074e-09,1.000000e+00
4,5.213648e-01,4.786352e-01,1.483967e-11
...,...,...,...
190,6.677842e-01,3.322158e-01,9.752275e-09
191,2.405064e-01,7.594936e-01,1.233132e-09
192,1.010262e-04,9.998989e-01,7.971936e-12
193,6.677842e-01,3.322158e-01,9.752275e-09


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p17s.csv", index = False,
         header=None)

In [76]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 149us/step - loss: 0.2318 - accuracy: 0.8830 - val_loss: 0.3139 - val_accuracy: 0.8769
Epoch 2/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2267 - accuracy: 0.8808 - val_loss: 0.3088 - val_accuracy: 0.8872
Epoch 3/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2291 - accuracy: 0.8830 - val_loss: 0.3119 - val_accuracy: 0.8769
Epoch 4/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2288 - accuracy: 0.8720 - val_loss: 0.3092 - val_accuracy: 0.8718
Epoch 5/1000
453/453 [==============================] - 0s 103us/step - loss: 0.2310 - accuracy: 0.8808 - val_loss: 0.3131 - val_accuracy: 0.8872
Epoch 6/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2379 - accuracy: 0.8808 - val_loss: 0.3105 - val_accuracy: 0.8923
Epoch 7/1000
453/453 [==============================] - 0s 104us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 153us/step - loss: 0.2271 - accuracy: 0.8852 - val_loss: 0.3109 - val_accuracy: 0.8872
Epoch 113/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2282 - accuracy: 0.8808 - val_loss: 0.3074 - val_accuracy: 0.8923
Epoch 114/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2305 - accuracy: 0.8918 - val_loss: 0.3149 - val_accuracy: 0.8872
Epoch 115/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2301 - accuracy: 0.8830 - val_loss: 0.3127 - val_accuracy: 0.8718
Epoch 116/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2254 - accuracy: 0.8764 - val_loss: 0.3048 - val_accuracy: 0.8872
Epoch 117/1000
453/453 [==============================] - 0s 160us/step - loss: 0.2319 - accuracy: 0.8874 - val_loss: 0.3130 - val_accuracy: 0.8667
Epoch 118/1000
453/453 [==============================] - 0s 178us/step - loss: 0.2300 - accuracy: 0.8742 - val_

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.130633). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


453/453 [==============================] - 0s 319us/step - loss: 0.2245 - accuracy: 0.8786 - val_loss: 0.3101 - val_accuracy: 0.8923
Epoch 161/1000
453/453 [==============================] - 0s 263us/step - loss: 0.2264 - accuracy: 0.8808 - val_loss: 0.3098 - val_accuracy: 0.8718
Epoch 162/1000
453/453 [==============================] - 0s 223us/step - loss: 0.2259 - accuracy: 0.8852 - val_loss: 0.3102 - val_accuracy: 0.8872
Epoch 163/1000
453/453 [==============================] - 0s 229us/step - loss: 0.2293 - accuracy: 0.8742 - val_loss: 0.3093 - val_accuracy: 0.8872
Epoch 164/1000
453/453 [==============================] - 0s 195us/step - loss: 0.2271 - accuracy: 0.8764 - val_loss: 0.3132 - val_accuracy: 0.8769
Epoch 165/1000
453/453 [==============================] - 0s 209us/step - loss: 0.2268 - accuracy: 0.8764 - val_loss: 0.3083 - val_accuracy: 0.8923
Epoch 166/1000
453/453 [==============================] - 0s 215us/step - loss: 0.2273 - accuracy: 0.8720 - val_loss: 0.3132 - 

453/453 [==============================] - 0s 203us/step - loss: 0.2250 - accuracy: 0.8830 - val_loss: 0.3072 - val_accuracy: 0.8769
Epoch 271/1000
453/453 [==============================] - 0s 133us/step - loss: 0.2269 - accuracy: 0.8786 - val_loss: 0.3079 - val_accuracy: 0.8872
Epoch 272/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2241 - accuracy: 0.8830 - val_loss: 0.3066 - val_accuracy: 0.8769
Epoch 273/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2257 - accuracy: 0.8830 - val_loss: 0.3051 - val_accuracy: 0.8923
Epoch 274/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2241 - accuracy: 0.8742 - val_loss: 0.3069 - val_accuracy: 0.8923
Epoch 275/1000
453/453 [==============================] - 0s 161us/step - loss: 0.2264 - accuracy: 0.8852 - val_loss: 0.3051 - val_accuracy: 0.8923
Epoch 276/1000
453/453 [==============================] - 0s 174us/step - loss: 0.2244 - accuracy: 0.8808 - val_loss: 0.3093 - 

...

453/453 [==============================] - 0s 199us/step - loss: 0.2236 - accuracy: 0.8830 - val_loss: 0.3178 - val_accuracy: 0.8872
Epoch 491/1000
453/453 [==============================] - 0s 176us/step - loss: 0.2260 - accuracy: 0.8852 - val_loss: 0.3133 - val_accuracy: 0.8974
Epoch 492/1000
453/453 [==============================] - 0s 171us/step - loss: 0.2230 - accuracy: 0.8830 - val_loss: 0.3167 - val_accuracy: 0.8872
Epoch 493/1000
453/453 [==============================] - 0s 188us/step - loss: 0.2246 - accuracy: 0.8786 - val_loss: 0.3172 - val_accuracy: 0.8923
Epoch 494/1000
453/453 [==============================] - 0s 231us/step - loss: 0.2265 - accuracy: 0.8808 - val_loss: 0.3167 - val_accuracy: 0.8872
Epoch 495/1000
453/453 [==============================] - 0s 302us/step - loss: 0.2236 - accuracy: 0.8852 - val_loss: 0.3162 - val_accuracy: 0.8872
Epoch 496/1000
453/453 [==============================] - 0s 207us/step - loss: 0.2227 - accuracy: 0.8830 - val_loss: 0.3191 - 

453/453 [==============================] - 0s 140us/step - loss: 0.2289 - accuracy: 0.8786 - val_loss: 0.3274 - val_accuracy: 0.8923
Epoch 601/1000
453/453 [==============================] - 0s 155us/step - loss: 0.2248 - accuracy: 0.8786 - val_loss: 0.3189 - val_accuracy: 0.8923
Epoch 602/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2273 - accuracy: 0.8808 - val_loss: 0.3196 - val_accuracy: 0.8872
Epoch 603/1000
453/453 [==============================] - 0s 225us/step - loss: 0.2258 - accuracy: 0.8808 - val_loss: 0.3200 - val_accuracy: 0.8872
Epoch 604/1000
453/453 [==============================] - 0s 253us/step - loss: 0.2250 - accuracy: 0.8808 - val_loss: 0.3192 - val_accuracy: 0.8872
Epoch 605/1000
453/453 [==============================] - 0s 249us/step - loss: 0.2248 - accuracy: 0.8786 - val_loss: 0.3227 - val_accuracy: 0.8872
Epoch 606/1000
453/453 [==============================] - 0s 240us/step - loss: 0.2248 - accuracy: 0.8852 - val_loss: 0.3230 - 

453/453 [==============================] - 0s 351us/step - loss: 0.2222 - accuracy: 0.8808 - val_loss: 0.3195 - val_accuracy: 0.8923
Epoch 711/1000
453/453 [==============================] - 0s 211us/step - loss: 0.2247 - accuracy: 0.8852 - val_loss: 0.3189 - val_accuracy: 0.8769
Epoch 712/1000
453/453 [==============================] - 0s 283us/step - loss: 0.2275 - accuracy: 0.8764 - val_loss: 0.3197 - val_accuracy: 0.8872
Epoch 713/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2247 - accuracy: 0.8764 - val_loss: 0.3166 - val_accuracy: 0.8821
Epoch 714/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2223 - accuracy: 0.8764 - val_loss: 0.3158 - val_accuracy: 0.8872
Epoch 715/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2244 - accuracy: 0.8808 - val_loss: 0.3172 - val_accuracy: 0.8872
Epoch 716/1000
453/453 [==============================] - 0s 135us/step - loss: 0.2248 - accuracy: 0.8742 - val_loss: 0.3188 - 

453/453 [==============================] - 0s 116us/step - loss: 0.2238 - accuracy: 0.8720 - val_loss: 0.3200 - val_accuracy: 0.8872
Epoch 821/1000
453/453 [==============================] - 0s 131us/step - loss: 0.2231 - accuracy: 0.8830 - val_loss: 0.3194 - val_accuracy: 0.8769
Epoch 822/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2222 - accuracy: 0.8808 - val_loss: 0.3203 - val_accuracy: 0.8872
Epoch 823/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2262 - accuracy: 0.8720 - val_loss: 0.3177 - val_accuracy: 0.8923
Epoch 824/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2244 - accuracy: 0.8874 - val_loss: 0.3187 - val_accuracy: 0.8872
Epoch 825/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2259 - accuracy: 0.8808 - val_loss: 0.3199 - val_accuracy: 0.8872
Epoch 826/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2248 - accuracy: 0.8852 - val_loss: 0.3214 - 

453/453 [==============================] - 0s 134us/step - loss: 0.2245 - accuracy: 0.8830 - val_loss: 0.3199 - val_accuracy: 0.8769
Epoch 931/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2228 - accuracy: 0.8764 - val_loss: 0.3179 - val_accuracy: 0.8923
Epoch 932/1000
453/453 [==============================] - 0s 188us/step - loss: 0.2239 - accuracy: 0.8764 - val_loss: 0.3167 - val_accuracy: 0.8872
Epoch 933/1000
453/453 [==============================] - 0s 154us/step - loss: 0.2254 - accuracy: 0.8742 - val_loss: 0.3179 - val_accuracy: 0.8872
Epoch 934/1000
453/453 [==============================] - 0s 177us/step - loss: 0.2228 - accuracy: 0.8830 - val_loss: 0.3155 - val_accuracy: 0.8923
Epoch 935/1000
453/453 [==============================] - 0s 178us/step - loss: 0.2239 - accuracy: 0.8830 - val_loss: 0.3187 - val_accuracy: 0.8769
Epoch 936/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2320 - accuracy: 0.8764 - val_loss: 0.3227 - 

In [77]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 88.00%


In [53]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [54]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [55]:
y_prob4 = df_proba4[df_proba4['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.87635960e-05, 9.99971300e-01, 9.79098100e-15],
       [1.24548726e-01, 8.75451270e-01, 3.46349100e-13],
       [9.17750100e-01, 8.22498700e-02, 5.38906140e-10],
       [3.31562380e-09, 4.96007370e-09, 1.00000000e+00],
       [5.21364800e-01, 4.78635160e-01, 1.48396750e-11],
       [3.31562380e-09, 4.96007370e-09, 1.00000000e+00],
       [1.38076260e-06, 9.99998570e-01, 0.00000000e+00],
       [2.40506400e-01, 7.59493600e-01, 1.23313160e-09],
       [3.03811450e-10, 5.95940600e-08, 9.99999900e-01],
       [3.31562380e-09, 4.96007370e-09, 1.00000000e+00],
       [3.03811450e-10, 5.95940600e-08, 9.99999900e-01],
       [5.21364800e-01, 4.78635160e-01, 1.48396750e-11],
       [6.44286930e-01, 3.55713070e-01, 4.28992500e-10],
       [4.17960100e-01, 5.82039800e-01, 1.29178620e-07],
       [2.79833970e-02, 9.72012160e-01, 4.44106500e-06],
       [3.03811450e-10, 5.95940600e-08, 9.99999900e-01],
       [3.03811450e-10, 5.95940600e-08, 9.99999900e-01],
       [3.31562380e-09, 4.96007

In [56]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9617357001972388

In [57]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9617357001972388

In [58]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9520118343195267

In [59]:
np.std(ovos)

0.012058510891320956

In [60]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9520118343195267

In [61]:
np.std(ovrs)

0.012058510891320956

In [78]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [79]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 85.77%


In [80]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.02275415165566839


In [81]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [82]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 89.12%


In [83]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.01007798


In [46]:
############ Feature selection using lasso ##########

In [62]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [63]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X_over[:,1:], y_over)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [64]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [65]:
X_train_features_over = np.vstack((names, X_over[:,1:]))
X_train_features_over = pd.DataFrame(X_train_features_over)

In [66]:
sel_features = X_train_features_over.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_over.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 101
selected features: 60


In [67]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 1,  3,  4,  5,  8,  9, 10, 12, 14, 19, 20, 22, 24, 25, 26, 28,
        31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 46, 47, 48, 49,
        50, 51, 52, 53, 56, 57, 58, 61, 62, 63, 65, 66, 67, 68, 72, 73,
        77, 78, 81, 82, 83, 85, 88, 89, 90, 94, 95, 96]])

In [68]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTAATACATAT', 'TTTATCTTTATGA', 'TTTAATTTAGTAAGT',
       'TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA',
       'TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA',
       'TTCAAGAAGGAGA', 'TTATTATGAGAATTACAAAAGCTACATAAATTAC',
       'TTATAAAAAGAAC',
       'TTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCTTTT',
       'TTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAG',
       'TGTTCCTGTTTTTATA',
       'TGGTATGCCTTTGTTTGTATAGTTTTCACTTCCACCTTTGGGAGTCTTTCCACTACCTATTTTGGTAGTAAGATTCCCTAACTTCTTCTCTTCCCATTCG',
       'TGGAGAAAGCAG', 'TGCTGCGGGTAG',
       'TGCAAAGACTTTTTTAGGTAACGGTAAGTCTAATGCCATCAATAAAATAGGCCAAATAATTGAGTGGAATCGCACAATTTCCTTAGCCATTAAATGAATA',
       'TGAAAACAAAGCT', 'TCTCCTTCTTGA',
       'TCCTTAAAAATGGTATGCCTTTGTTTGTATAGTTTTCACTTCCACCTTTGGGAGTCTTTCCACTACCTATTTTG

In [69]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [70]:
df_sel['strain'] = X.iloc[:,0]

In [71]:
df_sel

,TTTTAATACATAT,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACA,TTCAAGAAGGAGA,TTATTATGAGAATTACAAAAGCTACATAAATTAC,TTATAAAAAGAAC,TTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCTTTT,TTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAAG,...,CATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTTGCACATG,ATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCTTT,ATCCCATAGTGTGATAATTACTAT,ATATTTTAATAATTATT,ATATTTATGCAGT,AGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTACATTATTCATCT,AGATATTCATTTAATGGCTAAGGAAATTGTGCGATTCCACTCAATTATTTGGCCTATTTTATTGATGGCATTAGACTTACCGTTACCTAAAAAAGTCTTT,ACTTTCGAATT,pheno,strain
0,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,107
1,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,109
2,1,0,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,115
3,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,120335
4,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,1,SR4152
249,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,SR4153
250,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,SR4155
251,0,0,0,1,1,0,1,1,1,1,...,1,1,1,0,1,1,1,0,0,SR4156


In [72]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 61) (253,) (253, 62)


In [73]:
df_sel['pheno'].value_counts()

0    216
1     35
2      2
Name: pheno, dtype: int64

In [74]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 216), (1, 216), (2, 216)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [75]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel_over)

In [76]:
dat5 = pd.DataFrame(X_sel_test_over[:,-1])
dat5['test'] = y_sel_test_over

In [77]:
dat5

,0,test
0,312,1
1,BCH-SA-12,0
2,NRS209,2
3,CFBRSa29,0
4,NRS209,2
...,...,...
190,CA541,1
191,SR4152,1
192,NRS110,2
193,CFBRSa70,0


In [78]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [79]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [84]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 465us/step - loss: 1.0143 - accuracy: 0.5254 - val_loss: 0.8433 - val_accuracy: 0.7026
Epoch 2/1000
453/453 [==============================] - 0s 188us/step - loss: 0.7200 - accuracy: 0.7572 - val_loss: 0.6478 - val_accuracy: 0.7538
Epoch 3/1000
453/453 [==============================] - 0s 239us/step - loss: 0.5477 - accuracy: 0.8013 - val_loss: 0.5202 - val_accuracy: 0.7744
Epoch 4/1000
453/453 [==============================] - 0s 222us/step - loss: 0.4523 - accuracy: 0.8146 - val_loss: 0.4757 - val_accuracy: 0.7641
Epoch 5/1000
453/453 [==============================] - 0s 180us/step - loss: 0.4057 - accuracy: 0.8168 - val_loss: 0.4419 - val_accuracy: 0.7795
Epoch 6/1000
453/453 [==============================] - 0s 195us/step - loss: 0.3853 - accuracy: 0.8322 - val_loss: 0.4412 - val_accuracy: 0.7744
Epoch 7/1000
453/453 [==============================] - 0s 131us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 250us/step - loss: 0.2320 - accuracy: 0.8896 - val_loss: 0.3439 - val_accuracy: 0.8410
Epoch 113/1000
453/453 [==============================] - 0s 240us/step - loss: 0.2332 - accuracy: 0.8962 - val_loss: 0.3485 - val_accuracy: 0.8359
Epoch 114/1000
453/453 [==============================] - 0s 153us/step - loss: 0.2255 - accuracy: 0.8940 - val_loss: 0.3731 - val_accuracy: 0.8205
Epoch 115/1000
453/453 [==============================] - 0s 128us/step - loss: 0.2459 - accuracy: 0.8896 - val_loss: 0.3434 - val_accuracy: 0.8410
Epoch 116/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2297 - accuracy: 0.9007 - val_loss: 0.3432 - val_accuracy: 0.8462
Epoch 117/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2302 - accuracy: 0.9007 - val_loss: 0.3287 - val_accuracy: 0.8410
Epoch 118/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2288 - accuracy: 0.8962 - val_

453/453 [==============================] - 0s 126us/step - loss: 0.2181 - accuracy: 0.9029 - val_loss: 0.3637 - val_accuracy: 0.8410
Epoch 223/1000
453/453 [==============================] - 0s 196us/step - loss: 0.2209 - accuracy: 0.9007 - val_loss: 0.3592 - val_accuracy: 0.8410
Epoch 224/1000
453/453 [==============================] - 0s 215us/step - loss: 0.2168 - accuracy: 0.8962 - val_loss: 0.3649 - val_accuracy: 0.8513
Epoch 225/1000
453/453 [==============================] - 0s 239us/step - loss: 0.2186 - accuracy: 0.8985 - val_loss: 0.3584 - val_accuracy: 0.8410
Epoch 226/1000
453/453 [==============================] - 0s 272us/step - loss: 0.2174 - accuracy: 0.9007 - val_loss: 0.3645 - val_accuracy: 0.8410
Epoch 227/1000
453/453 [==============================] - 0s 385us/step - loss: 0.2172 - accuracy: 0.9007 - val_loss: 0.3653 - val_accuracy: 0.8410
Epoch 228/1000
453/453 [==============================] - 0s 349us/step - loss: 0.2201 - accuracy: 0.9007 - val_loss: 0.3606 - 

453/453 [==============================] - 0s 156us/step - loss: 0.2202 - accuracy: 0.8985 - val_loss: 0.3960 - val_accuracy: 0.8410
Epoch 333/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2235 - accuracy: 0.9051 - val_loss: 0.3909 - val_accuracy: 0.8410
Epoch 334/1000
453/453 [==============================] - 0s 136us/step - loss: 0.2141 - accuracy: 0.9029 - val_loss: 0.3821 - val_accuracy: 0.8410
Epoch 335/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2147 - accuracy: 0.9007 - val_loss: 0.3944 - val_accuracy: 0.8410
Epoch 336/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2168 - accuracy: 0.9029 - val_loss: 0.3872 - val_accuracy: 0.8410
Epoch 337/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2201 - accuracy: 0.9029 - val_loss: 0.3905 - val_accuracy: 0.8410
Epoch 338/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2215 - accuracy: 0.9029 - val_loss: 0.3796 - 

453/453 [==============================] - 0s 140us/step - loss: 0.2125 - accuracy: 0.9029 - val_loss: 0.4066 - val_accuracy: 0.8410
Epoch 443/1000
453/453 [==============================] - 0s 142us/step - loss: 0.2136 - accuracy: 0.9029 - val_loss: 0.4011 - val_accuracy: 0.8410
Epoch 444/1000
453/453 [==============================] - 0s 133us/step - loss: 0.2162 - accuracy: 0.9029 - val_loss: 0.4078 - val_accuracy: 0.8410
Epoch 445/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2132 - accuracy: 0.9029 - val_loss: 0.3977 - val_accuracy: 0.8410
Epoch 446/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2133 - accuracy: 0.9029 - val_loss: 0.4044 - val_accuracy: 0.8410
Epoch 447/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2131 - accuracy: 0.9029 - val_loss: 0.4078 - val_accuracy: 0.8410
Epoch 448/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2129 - accuracy: 0.9029 - val_loss: 0.4101 - 

453/453 [==============================] - 0s 230us/step - loss: 0.2117 - accuracy: 0.9029 - val_loss: 0.4528 - val_accuracy: 0.8410
Epoch 553/1000
453/453 [==============================] - 0s 209us/step - loss: 0.2105 - accuracy: 0.9029 - val_loss: 0.4500 - val_accuracy: 0.8410
Epoch 554/1000
453/453 [==============================] - 0s 188us/step - loss: 0.2127 - accuracy: 0.9029 - val_loss: 0.4459 - val_accuracy: 0.8410
Epoch 555/1000
453/453 [==============================] - 0s 255us/step - loss: 0.2150 - accuracy: 0.8985 - val_loss: 0.4451 - val_accuracy: 0.8410
Epoch 556/1000
453/453 [==============================] - 0s 363us/step - loss: 0.2153 - accuracy: 0.9029 - val_loss: 0.4402 - val_accuracy: 0.8410
Epoch 557/1000
453/453 [==============================] - 0s 186us/step - loss: 0.2126 - accuracy: 0.9029 - val_loss: 0.4431 - val_accuracy: 0.8410
Epoch 558/1000
453/453 [==============================] - 0s 220us/step - loss: 0.2145 - accuracy: 0.9029 - val_loss: 0.4479 - 

453/453 [==============================] - 0s 117us/step - loss: 0.2116 - accuracy: 0.9029 - val_loss: 0.4882 - val_accuracy: 0.8410
Epoch 663/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2111 - accuracy: 0.9029 - val_loss: 0.4923 - val_accuracy: 0.8410
Epoch 664/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2114 - accuracy: 0.9007 - val_loss: 0.4835 - val_accuracy: 0.8410
Epoch 665/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2111 - accuracy: 0.9029 - val_loss: 0.4778 - val_accuracy: 0.8410
Epoch 666/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2179 - accuracy: 0.9007 - val_loss: 0.5083 - val_accuracy: 0.8410
Epoch 667/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2139 - accuracy: 0.8940 - val_loss: 0.4637 - val_accuracy: 0.8410
Epoch 668/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2121 - accuracy: 0.9029 - val_loss: 0.4520 - 

453/453 [==============================] - 0s 114us/step - loss: 0.2116 - accuracy: 0.9029 - val_loss: 0.4952 - val_accuracy: 0.8410
Epoch 773/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2122 - accuracy: 0.9029 - val_loss: 0.5336 - val_accuracy: 0.8410
Epoch 774/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2115 - accuracy: 0.9029 - val_loss: 0.5206 - val_accuracy: 0.8410
Epoch 775/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2109 - accuracy: 0.9029 - val_loss: 0.5187 - val_accuracy: 0.8410
Epoch 776/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2112 - accuracy: 0.9029 - val_loss: 0.5318 - val_accuracy: 0.8410
Epoch 777/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2133 - accuracy: 0.9029 - val_loss: 0.5183 - val_accuracy: 0.8410
Epoch 778/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2104 - accuracy: 0.9029 - val_loss: 0.5324 - 

453/453 [==============================] - 0s 118us/step - loss: 0.2114 - accuracy: 0.9029 - val_loss: 0.4529 - val_accuracy: 0.8410
Epoch 883/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2095 - accuracy: 0.9029 - val_loss: 0.4521 - val_accuracy: 0.8410
Epoch 884/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2113 - accuracy: 0.9029 - val_loss: 0.4535 - val_accuracy: 0.8410
Epoch 885/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2114 - accuracy: 0.8962 - val_loss: 0.4521 - val_accuracy: 0.8410
Epoch 886/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2099 - accuracy: 0.8985 - val_loss: 0.4503 - val_accuracy: 0.8410
Epoch 887/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2110 - accuracy: 0.9029 - val_loss: 0.4487 - val_accuracy: 0.8410
Epoch 888/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2106 - accuracy: 0.9029 - val_loss: 0.4485 - 

453/453 [==============================] - 0s 117us/step - loss: 0.2089 - accuracy: 0.9029 - val_loss: 0.4807 - val_accuracy: 0.8410
Epoch 993/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2100 - accuracy: 0.9029 - val_loss: 0.4827 - val_accuracy: 0.8410
Epoch 994/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2116 - accuracy: 0.9029 - val_loss: 0.4705 - val_accuracy: 0.8410
Epoch 995/1000
453/453 [==============================] - 0s 128us/step - loss: 0.2111 - accuracy: 0.9029 - val_loss: 0.4675 - val_accuracy: 0.8410
Epoch 996/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2117 - accuracy: 0.9007 - val_loss: 0.4704 - val_accuracy: 0.8410
Epoch 997/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2111 - accuracy: 0.9029 - val_loss: 0.4687 - val_accuracy: 0.8410
Epoch 998/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2103 - accuracy: 0.9029 - val_loss: 0.4714 - 

In [120]:
acc_test2_over = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over*100))

195/195 [==============================] - 0s 61us/step
over-sampling test accuracy: 84.10%


In [86]:
pred5 = model2_over.predict_classes(X_sel_test_over)
pred5

array([0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2,
       1, 2, 2, 1, 0, 1, 2, 0, 1, 1, 2, 1, 0, 0, 2, 2, 2, 0, 1, 0, 2, 0,
       1, 1, 1, 1, 2, 2, 0, 2, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0, 2, 0, 1, 0,
       2, 2, 1, 0, 0, 0, 1, 0, 2, 2, 2, 2, 0, 1, 0, 2, 1, 0, 2, 0, 2, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 2, 2, 2, 0, 1, 0, 0,
       2, 1, 2, 1, 1, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 2, 2, 1, 0, 0, 1, 1,
       2, 0, 1, 0, 2, 1, 0, 2, 2, 0, 2, 1, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2,
       0, 0, 1, 0, 2, 0, 0, 1, 2, 2, 0, 0, 2, 2, 2, 2, 0, 1, 0, 1, 0, 1,
       2, 1, 0, 2, 1, 0, 1, 1, 0, 0, 0, 1, 2, 1, 1, 0, 2, 0, 1])

In [87]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,312,1,0
1,BCH-SA-12,0,0
2,NRS209,2,2
3,CFBRSa29,0,0
4,NRS209,2,2
...,...,...,...
190,CA541,1,1
191,SR4152,1,0
192,NRS110,2,2
193,CFBRSa70,0,0


In [88]:
proba5 = model2_over.predict_proba(X_sel_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [89]:
dat_proba5

,0,1,2
0,7.372800e-01,2.627200e-01,4.197748e-08
1,8.313845e-01,1.686155e-01,2.982275e-08
2,1.288949e-09,9.420666e-09,1.000000e+00
3,6.988474e-01,3.011526e-01,3.436671e-09
4,1.288949e-09,9.420666e-09,1.000000e+00
...,...,...,...
190,3.723218e-01,6.276781e-01,1.945911e-08
191,7.372800e-01,2.627200e-01,4.197748e-08
192,4.194510e-08,7.508231e-09,1.000000e+00
193,7.372800e-01,2.627200e-01,4.197748e-08


In [90]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [91]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p17s.csv", index = False,
         header=None)

In [124]:
hist2_over = model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 178us/step - loss: 0.2122 - accuracy: 0.9029 - val_loss: 0.6671 - val_accuracy: 0.8462
Epoch 2/1000
453/453 [==============================] - 0s 164us/step - loss: 0.2098 - accuracy: 0.9029 - val_loss: 0.6818 - val_accuracy: 0.8410
Epoch 3/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2103 - accuracy: 0.9029 - val_loss: 0.6914 - val_accuracy: 0.8410
Epoch 4/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2120 - accuracy: 0.9029 - val_loss: 0.6937 - val_accuracy: 0.8410
Epoch 5/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2119 - accuracy: 0.9029 - val_loss: 0.6901 - val_accuracy: 0.8359
Epoch 6/1000
453/453 [==============================] - 0s 140us/step - loss: 0.2104 - accuracy: 0.9029 - val_loss: 0.6860 - val_accuracy: 0.8359
Epoch 7/1000
453/453 [==============================] - 0s 156us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 231us/step - loss: 0.2097 - accuracy: 0.9029 - val_loss: 0.7639 - val_accuracy: 0.8359
Epoch 113/1000
453/453 [==============================] - 0s 287us/step - loss: 0.2094 - accuracy: 0.9029 - val_loss: 0.7596 - val_accuracy: 0.8410
Epoch 114/1000
453/453 [==============================] - 0s 379us/step - loss: 0.2095 - accuracy: 0.9029 - val_loss: 0.7654 - val_accuracy: 0.8359
Epoch 115/1000
453/453 [==============================] - 0s 189us/step - loss: 0.2085 - accuracy: 0.9029 - val_loss: 0.7638 - val_accuracy: 0.8410
Epoch 116/1000
453/453 [==============================] - 0s 385us/step - loss: 0.2095 - accuracy: 0.9029 - val_loss: 0.7628 - val_accuracy: 0.8410
Epoch 117/1000
453/453 [==============================] - 0s 256us/step - loss: 0.2091 - accuracy: 0.9029 - val_loss: 0.7575 - val_accuracy: 0.8410
Epoch 118/1000
453/453 [==============================] - 0s 327us/step - loss: 0.2092 - accuracy: 0.9029 - val_

453/453 [==============================] - 0s 223us/step - loss: 0.2110 - accuracy: 0.9029 - val_loss: 0.8278 - val_accuracy: 0.8359
Epoch 223/1000
453/453 [==============================] - 0s 132us/step - loss: 0.2097 - accuracy: 0.9029 - val_loss: 0.8222 - val_accuracy: 0.8359
Epoch 224/1000
453/453 [==============================] - 0s 172us/step - loss: 0.2086 - accuracy: 0.9029 - val_loss: 0.8167 - val_accuracy: 0.8359
Epoch 225/1000
453/453 [==============================] - 0s 176us/step - loss: 0.2101 - accuracy: 0.9029 - val_loss: 0.8230 - val_accuracy: 0.8359
Epoch 226/1000
453/453 [==============================] - 0s 180us/step - loss: 0.2099 - accuracy: 0.9029 - val_loss: 0.8203 - val_accuracy: 0.8359
Epoch 227/1000
453/453 [==============================] - 0s 156us/step - loss: 0.2091 - accuracy: 0.9029 - val_loss: 0.8160 - val_accuracy: 0.8359
Epoch 228/1000
453/453 [==============================] - 0s 351us/step - loss: 0.2087 - accuracy: 0.9029 - val_loss: 0.8234 - 

453/453 [==============================] - 0s 158us/step - loss: 0.2083 - accuracy: 0.9029 - val_loss: 0.8661 - val_accuracy: 0.8410
Epoch 333/1000
453/453 [==============================] - 0s 134us/step - loss: 0.2088 - accuracy: 0.9029 - val_loss: 0.8577 - val_accuracy: 0.8410
Epoch 334/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2086 - accuracy: 0.9029 - val_loss: 0.8621 - val_accuracy: 0.8410
Epoch 335/1000
453/453 [==============================] - 0s 153us/step - loss: 0.2088 - accuracy: 0.9029 - val_loss: 0.8611 - val_accuracy: 0.8410
Epoch 336/1000
453/453 [==============================] - 0s 193us/step - loss: 0.2098 - accuracy: 0.9029 - val_loss: 0.8620 - val_accuracy: 0.8410
Epoch 337/1000
453/453 [==============================] - 0s 135us/step - loss: 0.2078 - accuracy: 0.9029 - val_loss: 0.8639 - val_accuracy: 0.8410
Epoch 338/1000
453/453 [==============================] - 0s 133us/step - loss: 0.2098 - accuracy: 0.9029 - val_loss: 0.8668 - 

453/453 [==============================] - 0s 164us/step - loss: 0.2078 - accuracy: 0.9029 - val_loss: 0.8743 - val_accuracy: 0.8359
Epoch 443/1000
453/453 [==============================] - 0s 144us/step - loss: 0.2106 - accuracy: 0.8962 - val_loss: 0.8768 - val_accuracy: 0.8359
Epoch 444/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2076 - accuracy: 0.9029 - val_loss: 0.8746 - val_accuracy: 0.8359
Epoch 445/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2088 - accuracy: 0.9029 - val_loss: 0.8618 - val_accuracy: 0.8359
Epoch 446/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2087 - accuracy: 0.9029 - val_loss: 0.8570 - val_accuracy: 0.8410
Epoch 447/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2090 - accuracy: 0.9029 - val_loss: 0.8631 - val_accuracy: 0.8410
Epoch 448/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2088 - accuracy: 0.9029 - val_loss: 0.8573 - 

453/453 [==============================] - 0s 131us/step - loss: 0.2086 - accuracy: 0.9029 - val_loss: 0.9163 - val_accuracy: 0.8359
Epoch 553/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2101 - accuracy: 0.9029 - val_loss: 0.9161 - val_accuracy: 0.8359
Epoch 554/1000
453/453 [==============================] - 0s 156us/step - loss: 0.2082 - accuracy: 0.9029 - val_loss: 0.9197 - val_accuracy: 0.8359
Epoch 555/1000
453/453 [==============================] - 0s 128us/step - loss: 0.2086 - accuracy: 0.9029 - val_loss: 0.9213 - val_accuracy: 0.8359
Epoch 556/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2091 - accuracy: 0.9029 - val_loss: 0.9257 - val_accuracy: 0.8359
Epoch 557/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2085 - accuracy: 0.9029 - val_loss: 0.9259 - val_accuracy: 0.8359
Epoch 558/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2080 - accuracy: 0.9029 - val_loss: 0.9248 - 

453/453 [==============================] - 0s 112us/step - loss: 0.2089 - accuracy: 0.9029 - val_loss: 0.9374 - val_accuracy: 0.8410
Epoch 663/1000
453/453 [==============================] - 0s 143us/step - loss: 0.2073 - accuracy: 0.9029 - val_loss: 0.9411 - val_accuracy: 0.8410
Epoch 664/1000
453/453 [==============================] - 0s 131us/step - loss: 0.2081 - accuracy: 0.9029 - val_loss: 0.9411 - val_accuracy: 0.8410
Epoch 665/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2085 - accuracy: 0.9029 - val_loss: 0.9479 - val_accuracy: 0.8410
Epoch 666/1000
453/453 [==============================] - 0s 128us/step - loss: 0.2091 - accuracy: 0.9029 - val_loss: 0.9448 - val_accuracy: 0.8410
Epoch 667/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2080 - accuracy: 0.9029 - val_loss: 0.9461 - val_accuracy: 0.8410
Epoch 668/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2074 - accuracy: 0.9029 - val_loss: 0.9467 - 

453/453 [==============================] - 0s 106us/step - loss: 0.2080 - accuracy: 0.9029 - val_loss: 0.9519 - val_accuracy: 0.8359
Epoch 773/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2076 - accuracy: 0.9029 - val_loss: 0.9533 - val_accuracy: 0.8410
Epoch 774/1000
453/453 [==============================] - 0s 142us/step - loss: 0.2076 - accuracy: 0.9029 - val_loss: 0.9628 - val_accuracy: 0.8410
Epoch 775/1000
453/453 [==============================] - 0s 146us/step - loss: 0.2072 - accuracy: 0.9029 - val_loss: 0.9672 - val_accuracy: 0.8359
Epoch 776/1000
453/453 [==============================] - 0s 142us/step - loss: 0.2075 - accuracy: 0.9029 - val_loss: 0.9679 - val_accuracy: 0.8359
Epoch 777/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2080 - accuracy: 0.9029 - val_loss: 0.9706 - val_accuracy: 0.8359
Epoch 778/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2076 - accuracy: 0.9029 - val_loss: 0.9683 - 

453/453 [==============================] - 0s 109us/step - loss: 0.2077 - accuracy: 0.9029 - val_loss: 1.0268 - val_accuracy: 0.8410
Epoch 883/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2085 - accuracy: 0.9029 - val_loss: 1.0289 - val_accuracy: 0.8410
Epoch 884/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2073 - accuracy: 0.9029 - val_loss: 1.0189 - val_accuracy: 0.8410
Epoch 885/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2100 - accuracy: 0.8962 - val_loss: 1.0219 - val_accuracy: 0.8410
Epoch 886/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2079 - accuracy: 0.9029 - val_loss: 1.0127 - val_accuracy: 0.8410
Epoch 887/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2085 - accuracy: 0.9029 - val_loss: 1.0192 - val_accuracy: 0.8410
Epoch 888/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2086 - accuracy: 0.9029 - val_loss: 1.0194 - 

453/453 [==============================] - 0s 116us/step - loss: 0.2074 - accuracy: 0.9029 - val_loss: 1.0765 - val_accuracy: 0.8359
Epoch 993/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2079 - accuracy: 0.9029 - val_loss: 1.0677 - val_accuracy: 0.8359
Epoch 994/1000
453/453 [==============================] - 0s 136us/step - loss: 0.2169 - accuracy: 0.9007 - val_loss: 1.0916 - val_accuracy: 0.8359
Epoch 995/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2215 - accuracy: 0.8940 - val_loss: 0.9710 - val_accuracy: 0.8359
Epoch 996/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2162 - accuracy: 0.8962 - val_loss: 1.2147 - val_accuracy: 0.8308
Epoch 997/1000
453/453 [==============================] - 0s 174us/step - loss: 0.2121 - accuracy: 0.9007 - val_loss: 1.1810 - val_accuracy: 0.8359
Epoch 998/1000
453/453 [==============================] - 0s 135us/step - loss: 0.2097 - accuracy: 0.8962 - val_loss: 1.1055 - 

In [125]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over.history['accuracy'])*100))

over-sampling train accuracy: 90.25%


In [80]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [81]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS245,1,2,1.345807e-02,2.164788e-01,7.700630e-01
1,p0006kpresabs_qual,NY439,2,2,2.674153e-02,9.294230e-04,9.723290e-01
2,p0006kpresabs_qual,CA544,1,0,4.147484e-01,3.626331e-01,2.226184e-01
3,p0006kpresabs_qual,CA541,2,0,4.147484e-01,3.626331e-01,2.226184e-01
4,p0006kpresabs_qual,EUH15,1,0,4.147484e-01,3.626331e-01,2.226184e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CA541,1,1,3.723218e-01,6.276781e-01,1.945911e-08
985,p0017Skpresabs_qual,SR4152,1,0,7.372800e-01,2.627200e-01,4.197748e-08
986,p0017Skpresabs_qual,NRS110,2,2,4.194510e-08,7.508231e-09,1.000000e+00
987,p0017Skpresabs_qual,CFBRSa70,0,0,7.372800e-01,2.627200e-01,4.197748e-08


In [82]:
y_prob5 = df_proba5[df_proba5['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[7.37280000e-01, 2.62720000e-01, 4.19774800e-08],
       [8.31384500e-01, 1.68615500e-01, 2.98227470e-08],
       [1.28894920e-09, 9.42066600e-09, 1.00000000e+00],
       [6.98847400e-01, 3.01152560e-01, 3.43667070e-09],
       [1.28894920e-09, 9.42066600e-09, 1.00000000e+00],
       [7.37280000e-01, 2.62720000e-01, 4.19774800e-08],
       [7.37280000e-01, 2.62720000e-01, 4.19774800e-08],
       [4.19451030e-08, 7.50823100e-09, 1.00000000e+00],
       [8.31384500e-01, 1.68615500e-01, 2.98227470e-08],
       [1.28894920e-09, 9.42066600e-09, 1.00000000e+00],
       [4.19451030e-08, 7.50823100e-09, 1.00000000e+00],
       [4.19451030e-08, 7.50823100e-09, 1.00000000e+00],
       [4.19451030e-08, 7.50823100e-09, 1.00000000e+00],
       [4.19451030e-08, 7.50823100e-09, 1.00000000e+00],
       [7.37280000e-01, 2.62720000e-01, 4.19774800e-08],
       [4.19451030e-08, 7.50823100e-09, 1.00000000e+00],
       [1.28894920e-09, 9.42066600e-09, 1.00000000e+00],
       [6.98847400e-01, 3.01152

In [83]:
ovo5 = rocauc_ovo(y_sel_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9471005917159764

In [84]:
ovr5 = rocauc_ovr(y_sel_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9471005917159764

In [85]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel_over)

In [86]:
dat6 = pd.DataFrame(X_sel_test_over[:,-1])
dat6['test'] = y_sel_test_over

In [87]:
dat6

,0,test
0,CFBREBSa119,0
1,NRS001,1
2,NRS074,0
3,NRS209,2
4,GA231,0
...,...,...
190,NRS252,0
191,SR2852,1
192,NRS108,1
193,NRS202,0


In [88]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [89]:
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [97]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 475us/step - loss: 1.0587 - accuracy: 0.3996 - val_loss: 0.8505 - val_accuracy: 0.6051
Epoch 2/1000
453/453 [==============================] - 0s 176us/step - loss: 0.7732 - accuracy: 0.7064 - val_loss: 0.6637 - val_accuracy: 0.7692
Epoch 3/1000
453/453 [==============================] - 0s 210us/step - loss: 0.6416 - accuracy: 0.7903 - val_loss: 0.5771 - val_accuracy: 0.7846
Epoch 4/1000
453/453 [==============================] - 0s 194us/step - loss: 0.5760 - accuracy: 0.7682 - val_loss: 0.5196 - val_accuracy: 0.8564
Epoch 5/1000
453/453 [==============================] - 0s 201us/step - loss: 0.5138 - accuracy: 0.7837 - val_loss: 0.4645 - val_accuracy: 0.8051
Epoch 6/1000
453/453 [==============================] - 0s 157us/step - loss: 0.4688 - accuracy: 0.7991 - val_loss: 0.4320 - val_accuracy: 0.8103
Epoch 7/1000
453/453 [==============================] - 0s 145us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 138us/step - loss: 0.2963 - accuracy: 0.8389 - val_loss: 0.2814 - val_accuracy: 0.8615
Epoch 113/1000
453/453 [==============================] - 0s 132us/step - loss: 0.2658 - accuracy: 0.8720 - val_loss: 0.2854 - val_accuracy: 0.8564
Epoch 114/1000
453/453 [==============================] - 0s 183us/step - loss: 0.2647 - accuracy: 0.8720 - val_loss: 0.2824 - val_accuracy: 0.8615
Epoch 115/1000
453/453 [==============================] - 0s 134us/step - loss: 0.2705 - accuracy: 0.8675 - val_loss: 0.2809 - val_accuracy: 0.8615
Epoch 116/1000
453/453 [==============================] - 0s 99us/step - loss: 0.2686 - accuracy: 0.8675 - val_loss: 0.2871 - val_accuracy: 0.8667
Epoch 117/1000
453/453 [==============================] - 0s 96us/step - loss: 0.2671 - accuracy: 0.8786 - val_loss: 0.2860 - val_accuracy: 0.8410
Epoch 118/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2636 - accuracy: 0.8587 - val_lo

453/453 [==============================] - 0s 140us/step - loss: 0.2499 - accuracy: 0.8742 - val_loss: 0.2732 - val_accuracy: 0.8667
Epoch 223/1000
453/453 [==============================] - 0s 139us/step - loss: 0.2528 - accuracy: 0.8675 - val_loss: 0.2750 - val_accuracy: 0.8564
Epoch 224/1000
453/453 [==============================] - 0s 166us/step - loss: 0.2542 - accuracy: 0.8698 - val_loss: 0.2810 - val_accuracy: 0.8615
Epoch 225/1000
453/453 [==============================] - 0s 132us/step - loss: 0.2512 - accuracy: 0.8720 - val_loss: 0.2752 - val_accuracy: 0.8615
Epoch 226/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2530 - accuracy: 0.8675 - val_loss: 0.2750 - val_accuracy: 0.8615
Epoch 227/1000
453/453 [==============================] - 0s 138us/step - loss: 0.2499 - accuracy: 0.8764 - val_loss: 0.2781 - val_accuracy: 0.8615
Epoch 228/1000
453/453 [==============================] - 0s 133us/step - loss: 0.2535 - accuracy: 0.8698 - val_loss: 0.2964 - 

453/453 [==============================] - 0s 91us/step - loss: 0.2439 - accuracy: 0.8720 - val_loss: 0.2808 - val_accuracy: 0.8513
Epoch 333/1000
453/453 [==============================] - 0s 89us/step - loss: 0.2559 - accuracy: 0.8631 - val_loss: 0.2845 - val_accuracy: 0.8564
Epoch 334/1000
453/453 [==============================] - 0s 237us/step - loss: 0.2509 - accuracy: 0.8653 - val_loss: 0.2721 - val_accuracy: 0.8667
Epoch 335/1000
453/453 [==============================] - 0s 175us/step - loss: 0.2428 - accuracy: 0.8786 - val_loss: 0.2699 - val_accuracy: 0.8667
Epoch 336/1000
453/453 [==============================] - 0s 137us/step - loss: 0.2460 - accuracy: 0.8720 - val_loss: 0.2709 - val_accuracy: 0.8615
Epoch 337/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2418 - accuracy: 0.8764 - val_loss: 0.2700 - val_accuracy: 0.8667
Epoch 338/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2444 - accuracy: 0.8830 - val_loss: 0.2752 - va

453/453 [==============================] - 0s 116us/step - loss: 0.2408 - accuracy: 0.8808 - val_loss: 0.2736 - val_accuracy: 0.8667
Epoch 443/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2507 - accuracy: 0.8653 - val_loss: 0.2656 - val_accuracy: 0.8667
Epoch 444/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2395 - accuracy: 0.8786 - val_loss: 0.2657 - val_accuracy: 0.8615
Epoch 445/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2425 - accuracy: 0.8808 - val_loss: 0.2670 - val_accuracy: 0.8615
Epoch 446/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2404 - accuracy: 0.8786 - val_loss: 0.2724 - val_accuracy: 0.8667
Epoch 447/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2426 - accuracy: 0.8786 - val_loss: 0.2667 - val_accuracy: 0.8615
Epoch 448/1000
453/453 [==============================] - 0s 122us/step - loss: 0.2385 - accuracy: 0.8808 - val_loss: 0.2676 - 

453/453 [==============================] - 0s 114us/step - loss: 0.2349 - accuracy: 0.8786 - val_loss: 0.2621 - val_accuracy: 0.8615
Epoch 553/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2360 - accuracy: 0.8764 - val_loss: 0.2623 - val_accuracy: 0.8667
Epoch 554/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2357 - accuracy: 0.8808 - val_loss: 0.2700 - val_accuracy: 0.8923
Epoch 555/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2409 - accuracy: 0.8675 - val_loss: 0.2630 - val_accuracy: 0.8667
Epoch 556/1000
453/453 [==============================] - 0s 133us/step - loss: 0.2356 - accuracy: 0.8830 - val_loss: 0.2652 - val_accuracy: 0.8615
Epoch 557/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2350 - accuracy: 0.8874 - val_loss: 0.2788 - val_accuracy: 0.8923
Epoch 558/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2403 - accuracy: 0.8698 - val_loss: 0.2740 - 

453/453 [==============================] - 0s 111us/step - loss: 0.2348 - accuracy: 0.8786 - val_loss: 0.2606 - val_accuracy: 0.8667
Epoch 663/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2346 - accuracy: 0.8786 - val_loss: 0.2635 - val_accuracy: 0.8923
Epoch 664/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2363 - accuracy: 0.8852 - val_loss: 0.2610 - val_accuracy: 0.8615
Epoch 665/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2334 - accuracy: 0.8764 - val_loss: 0.2619 - val_accuracy: 0.8615
Epoch 666/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2359 - accuracy: 0.8742 - val_loss: 0.2615 - val_accuracy: 0.8615
Epoch 667/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2345 - accuracy: 0.8764 - val_loss: 0.2627 - val_accuracy: 0.8667
Epoch 668/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2366 - accuracy: 0.8742 - val_loss: 0.2642 - 

453/453 [==============================] - 0s 107us/step - loss: 0.2342 - accuracy: 0.8808 - val_loss: 0.2607 - val_accuracy: 0.8667
Epoch 773/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2342 - accuracy: 0.8786 - val_loss: 0.2579 - val_accuracy: 0.8615
Epoch 774/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2348 - accuracy: 0.8786 - val_loss: 0.2617 - val_accuracy: 0.8667
Epoch 775/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2318 - accuracy: 0.8808 - val_loss: 0.2602 - val_accuracy: 0.8615
Epoch 776/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2362 - accuracy: 0.8675 - val_loss: 0.2605 - val_accuracy: 0.8615
Epoch 777/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2338 - accuracy: 0.8808 - val_loss: 0.2607 - val_accuracy: 0.8667
Epoch 778/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2367 - accuracy: 0.8742 - val_loss: 0.2611 - 

453/453 [==============================] - 0s 195us/step - loss: 0.2319 - accuracy: 0.8808 - val_loss: 0.2553 - val_accuracy: 0.8667
Epoch 883/1000
453/453 [==============================] - 0s 149us/step - loss: 0.2303 - accuracy: 0.8742 - val_loss: 0.2551 - val_accuracy: 0.8667
Epoch 884/1000
453/453 [==============================] - 0s 153us/step - loss: 0.2336 - accuracy: 0.8786 - val_loss: 0.2567 - val_accuracy: 0.8667
Epoch 885/1000
453/453 [==============================] - 0s 132us/step - loss: 0.2367 - accuracy: 0.8808 - val_loss: 0.2573 - val_accuracy: 0.8667
Epoch 886/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2355 - accuracy: 0.8720 - val_loss: 0.2555 - val_accuracy: 0.8667
Epoch 887/1000
453/453 [==============================] - 0s 149us/step - loss: 0.2341 - accuracy: 0.8786 - val_loss: 0.2572 - val_accuracy: 0.8667
Epoch 888/1000
453/453 [==============================] - 0s 181us/step - loss: 0.2335 - accuracy: 0.8720 - val_loss: 0.2608 - 

453/453 [==============================] - 0s 117us/step - loss: 0.2319 - accuracy: 0.8764 - val_loss: 0.2584 - val_accuracy: 0.8667
Epoch 993/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2331 - accuracy: 0.8786 - val_loss: 0.2568 - val_accuracy: 0.8667
Epoch 994/1000
453/453 [==============================] - 0s 145us/step - loss: 0.2331 - accuracy: 0.8808 - val_loss: 0.2560 - val_accuracy: 0.8974
Epoch 995/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2370 - accuracy: 0.8764 - val_loss: 0.2538 - val_accuracy: 0.8667
Epoch 996/1000
453/453 [==============================] - 0s 131us/step - loss: 0.2332 - accuracy: 0.8698 - val_loss: 0.2516 - val_accuracy: 0.8718
Epoch 997/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2334 - accuracy: 0.8675 - val_loss: 0.2549 - val_accuracy: 0.8667
Epoch 998/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2330 - accuracy: 0.8786 - val_loss: 0.2540 - 

In [141]:
acc_test2_over2 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over2*100))

195/195 [==============================] - 0s 55us/step
over-sampling test accuracy: 87.18%


In [99]:
pred6 = model2_over2.predict_classes(X_sel_test_over)
pred6

array([0, 1, 0, 2, 1, 2, 0, 1, 1, 1, 2, 0, 2, 0, 0, 0, 2, 2, 2, 1, 2, 1,
       0, 0, 0, 1, 0, 0, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 1, 0, 1, 1,
       2, 2, 2, 0, 1, 0, 1, 2, 0, 0, 1, 0, 1, 0, 2, 0, 2, 0, 1, 0, 0, 1,
       1, 2, 2, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 2, 0, 0, 0, 1, 0,
       1, 2, 1, 0, 2, 2, 1, 0, 0, 2, 0, 2, 0, 1, 2, 2, 1, 0, 0, 1, 0, 2,
       2, 1, 0, 1, 2, 0, 0, 0, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 2, 0, 1, 0,
       0, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 2,
       2, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1,
       0, 2, 2, 2, 1, 0, 2, 1, 1, 2, 2, 0, 2, 1, 0, 1, 1, 0, 2])

In [100]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,CFBREBSa119,0,0
1,NRS001,1,1
2,NRS074,0,0
3,NRS209,2,2
4,GA231,0,1
...,...,...,...
190,NRS252,0,0
191,SR2852,1,1
192,NRS108,1,1
193,NRS202,0,0


In [101]:
proba6 = model2_over2.predict_proba(X_sel_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [102]:
dat_proba6

,0,1,2
0,6.888959e-01,3.111041e-01,2.228958e-09
1,4.984043e-01,5.015957e-01,3.714970e-10
2,6.888959e-01,3.111041e-01,2.228958e-09
3,1.076489e-09,4.213780e-08,1.000000e+00
4,3.451807e-01,6.548194e-01,4.039059e-09
...,...,...,...
190,7.239556e-01,2.760444e-01,1.176030e-09
191,1.052275e-07,9.999999e-01,1.101559e-28
192,1.540350e-17,1.000000e+00,9.011977e-16
193,6.888959e-01,3.111041e-01,2.228958e-09


In [103]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [104]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p17s.csv", index = False,
         header=None)

In [145]:
hist2_over2 = model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 290us/step - loss: 0.2323 - accuracy: 0.8786 - val_loss: 0.2468 - val_accuracy: 0.8667
Epoch 2/1000
453/453 [==============================] - 0s 250us/step - loss: 0.2328 - accuracy: 0.8808 - val_loss: 0.2485 - val_accuracy: 0.8667
Epoch 3/1000
453/453 [==============================] - 0s 175us/step - loss: 0.2318 - accuracy: 0.8653 - val_loss: 0.2485 - val_accuracy: 0.8718
Epoch 4/1000
453/453 [==============================] - 0s 152us/step - loss: 0.2323 - accuracy: 0.8786 - val_loss: 0.2482 - val_accuracy: 0.8718
Epoch 5/1000
453/453 [==============================] - 0s 137us/step - loss: 0.2302 - accuracy: 0.8764 - val_loss: 0.2457 - val_accuracy: 0.8667
Epoch 6/1000
453/453 [==============================] - 0s 146us/step - loss: 0.2324 - accuracy: 0.8786 - val_loss: 0.2468 - val_accuracy: 0.8718
Epoch 7/1000
453/453 [==============================] - 0s 141us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 178us/step - loss: 0.2307 - accuracy: 0.8808 - val_loss: 0.2460 - val_accuracy: 0.8667
Epoch 113/1000
453/453 [==============================] - 0s 220us/step - loss: 0.2318 - accuracy: 0.8786 - val_loss: 0.2502 - val_accuracy: 0.8718
Epoch 114/1000
453/453 [==============================] - 0s 237us/step - loss: 0.2319 - accuracy: 0.8764 - val_loss: 0.2451 - val_accuracy: 0.8718
Epoch 115/1000
453/453 [==============================] - 0s 299us/step - loss: 0.2323 - accuracy: 0.8764 - val_loss: 0.2453 - val_accuracy: 0.8718
Epoch 116/1000
453/453 [==============================] - 0s 330us/step - loss: 0.2313 - accuracy: 0.8808 - val_loss: 0.2421 - val_accuracy: 0.8667
Epoch 117/1000
453/453 [==============================] - 0s 276us/step - loss: 0.2327 - accuracy: 0.8742 - val_loss: 0.2452 - val_accuracy: 0.8667
Epoch 118/1000
453/453 [==============================] - 0s 317us/step - loss: 0.2326 - accuracy: 0.8808 - val_

453/453 [==============================] - 0s 118us/step - loss: 0.2305 - accuracy: 0.8786 - val_loss: 0.2465 - val_accuracy: 0.8769
Epoch 223/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2310 - accuracy: 0.8808 - val_loss: 0.2440 - val_accuracy: 0.8667
Epoch 224/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2313 - accuracy: 0.8786 - val_loss: 0.2451 - val_accuracy: 0.8718
Epoch 225/1000
453/453 [==============================] - 0s 142us/step - loss: 0.2328 - accuracy: 0.8764 - val_loss: 0.2477 - val_accuracy: 0.8769
Epoch 226/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2331 - accuracy: 0.8742 - val_loss: 0.2432 - val_accuracy: 0.8667
Epoch 227/1000
453/453 [==============================] - 0s 205us/step - loss: 0.2282 - accuracy: 0.8808 - val_loss: 0.2451 - val_accuracy: 0.8718
Epoch 228/1000
453/453 [==============================] - 0s 256us/step - loss: 0.2298 - accuracy: 0.8786 - val_loss: 0.2462 - 

453/453 [==============================] - 0s 117us/step - loss: 0.2310 - accuracy: 0.8742 - val_loss: 0.2451 - val_accuracy: 0.8667
Epoch 333/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2316 - accuracy: 0.8808 - val_loss: 0.2452 - val_accuracy: 0.8667
Epoch 334/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2355 - accuracy: 0.8808 - val_loss: 0.2541 - val_accuracy: 0.8667
Epoch 335/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2343 - accuracy: 0.8720 - val_loss: 0.2455 - val_accuracy: 0.8718
Epoch 336/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2315 - accuracy: 0.8764 - val_loss: 0.2473 - val_accuracy: 0.8718
Epoch 337/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2316 - accuracy: 0.8786 - val_loss: 0.2462 - val_accuracy: 0.8718
Epoch 338/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2315 - accuracy: 0.8764 - val_loss: 0.2435 - 

453/453 [==============================] - 0s 110us/step - loss: 0.2308 - accuracy: 0.8720 - val_loss: 0.2448 - val_accuracy: 0.8718
Epoch 443/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2315 - accuracy: 0.8808 - val_loss: 0.2428 - val_accuracy: 0.8769
Epoch 444/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2314 - accuracy: 0.8808 - val_loss: 0.2428 - val_accuracy: 0.8769
Epoch 445/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2307 - accuracy: 0.8764 - val_loss: 0.2451 - val_accuracy: 0.8769
Epoch 446/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2353 - accuracy: 0.8764 - val_loss: 0.2472 - val_accuracy: 0.8718
Epoch 447/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2338 - accuracy: 0.8720 - val_loss: 0.2452 - val_accuracy: 0.8718
Epoch 448/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2313 - accuracy: 0.8808 - val_loss: 0.2431 - 

453/453 [==============================] - 0s 105us/step - loss: 0.2297 - accuracy: 0.8808 - val_loss: 0.2451 - val_accuracy: 0.8718
Epoch 553/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2289 - accuracy: 0.8808 - val_loss: 0.2434 - val_accuracy: 0.8718
Epoch 554/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2295 - accuracy: 0.8742 - val_loss: 0.2412 - val_accuracy: 0.8769
Epoch 555/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2304 - accuracy: 0.8786 - val_loss: 0.2437 - val_accuracy: 0.8821
Epoch 556/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2305 - accuracy: 0.8786 - val_loss: 0.2428 - val_accuracy: 0.8769
Epoch 557/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2318 - accuracy: 0.8764 - val_loss: 0.2436 - val_accuracy: 0.8821
Epoch 558/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2301 - accuracy: 0.8786 - val_loss: 0.2407 - 

Epoch 662/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2288 - accuracy: 0.8808 - val_loss: 0.2405 - val_accuracy: 0.8769
Epoch 663/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2314 - accuracy: 0.8653 - val_loss: 0.2413 - val_accuracy: 0.8769
Epoch 664/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2304 - accuracy: 0.8808 - val_loss: 0.2403 - val_accuracy: 0.8718
Epoch 665/1000
453/453 [==============================] - 0s 158us/step - loss: 0.2299 - accuracy: 0.8786 - val_loss: 0.2414 - val_accuracy: 0.8718
Epoch 666/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2289 - accuracy: 0.8786 - val_loss: 0.2414 - val_accuracy: 0.8718
Epoch 667/1000
453/453 [==============================] - 0s 128us/step - loss: 0.2314 - accuracy: 0.8742 - val_loss: 0.2436 - val_accuracy: 0.8769
Epoch 668/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2329 - accuracy: 0.8830 - val_

453/453 [==============================] - 0s 113us/step - loss: 0.2288 - accuracy: 0.8808 - val_loss: 0.2433 - val_accuracy: 0.8718
Epoch 773/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2297 - accuracy: 0.8808 - val_loss: 0.2432 - val_accuracy: 0.8718
Epoch 774/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2297 - accuracy: 0.8698 - val_loss: 0.2420 - val_accuracy: 0.8769
Epoch 775/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2297 - accuracy: 0.8808 - val_loss: 0.2416 - val_accuracy: 0.8718
Epoch 776/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2304 - accuracy: 0.8808 - val_loss: 0.2421 - val_accuracy: 0.8718
Epoch 777/1000
453/453 [==============================] - 0s 135us/step - loss: 0.2296 - accuracy: 0.8786 - val_loss: 0.2418 - val_accuracy: 0.8769
Epoch 778/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2303 - accuracy: 0.8698 - val_loss: 0.2413 - 

453/453 [==============================] - 0s 107us/step - loss: 0.2289 - accuracy: 0.8808 - val_loss: 0.2447 - val_accuracy: 0.8718
Epoch 883/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2323 - accuracy: 0.8786 - val_loss: 0.2457 - val_accuracy: 0.8769
Epoch 884/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2306 - accuracy: 0.8808 - val_loss: 0.2432 - val_accuracy: 0.8769
Epoch 885/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2293 - accuracy: 0.8808 - val_loss: 0.2443 - val_accuracy: 0.8769
Epoch 886/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2307 - accuracy: 0.8808 - val_loss: 0.2437 - val_accuracy: 0.8718
Epoch 887/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2278 - accuracy: 0.8808 - val_loss: 0.2471 - val_accuracy: 0.8769
Epoch 888/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2333 - accuracy: 0.8653 - val_loss: 0.2438 - 

453/453 [==============================] - 0s 235us/step - loss: 0.2311 - accuracy: 0.8764 - val_loss: 0.2450 - val_accuracy: 0.8769
Epoch 993/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2294 - accuracy: 0.8786 - val_loss: 0.2447 - val_accuracy: 0.8718
Epoch 994/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2286 - accuracy: 0.8786 - val_loss: 0.2438 - val_accuracy: 0.8718
Epoch 995/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2296 - accuracy: 0.8786 - val_loss: 0.2430 - val_accuracy: 0.8718
Epoch 996/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2295 - accuracy: 0.8786 - val_loss: 0.2446 - val_accuracy: 0.8769
Epoch 997/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2299 - accuracy: 0.8720 - val_loss: 0.2429 - val_accuracy: 0.8718
Epoch 998/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2286 - accuracy: 0.8808 - val_loss: 0.2426 - 

In [146]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over2.history['accuracy'])*100))

over-sampling train accuracy: 87.74%


In [90]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [91]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS249,2,1,1.888869e-01,5.108038e-01,3.003094e-01
1,p0006kpresabs_qual,NRS188,1,1,1.888869e-01,5.108038e-01,3.003094e-01
2,p0006kpresabs_qual,NRS232,2,2,4.222906e-01,7.029924e-02,5.074101e-01
3,p0006kpresabs_qual,NY439,2,2,3.558408e-04,2.976018e-04,9.993465e-01
4,p0006kpresabs_qual,GA27,2,1,3.940971e-01,4.184215e-01,1.874814e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS252,0,0,7.239556e-01,2.760444e-01,1.176030e-09
985,p0017Skpresabs_qual,SR2852,1,1,1.052276e-07,9.999999e-01,1.101559e-28
986,p0017Skpresabs_qual,NRS108,1,1,1.540350e-17,1.000000e+00,9.011977e-16
987,p0017Skpresabs_qual,NRS202,0,0,6.888959e-01,3.111042e-01,2.228958e-09


In [92]:
y_prob6 = df_proba6[df_proba6['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[6.88895900e-01, 3.11104150e-01, 2.22895800e-09],
       [4.98404260e-01, 5.01595740e-01, 3.71497030e-10],
       [6.88895900e-01, 3.11104150e-01, 2.22895800e-09],
       [1.07648910e-09, 4.21377950e-08, 1.00000000e+00],
       [3.45180700e-01, 6.54819370e-01, 4.03905930e-09],
       [1.09771880e-09, 4.40465500e-08, 1.00000000e+00],
       [6.88895900e-01, 3.11104150e-01, 2.22895800e-09],
       [2.77232640e-04, 9.99722800e-01, 2.69030090e-09],
       [8.64035040e-02, 9.13596450e-01, 3.01891400e-10],
       [6.98568860e-03, 9.93014340e-01, 1.54115170e-09],
       [1.07648910e-09, 4.21377950e-08, 1.00000000e+00],
       [5.71686500e-01, 4.28313500e-01, 6.65629000e-12],
       [1.09771880e-09, 4.40465500e-08, 1.00000000e+00],
       [9.99813850e-01, 1.86203220e-04, 9.17890300e-16],
       [7.23955600e-01, 2.76044400e-01, 1.17602990e-09],
       [6.88895900e-01, 3.11104150e-01, 2.22895800e-09],
       [1.09771880e-09, 4.40465500e-08, 1.00000000e+00],
       [1.07648910e-09, 4.21377

In [94]:
ovo6 = rocauc_ovo(y_sel_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.969664694280079

In [95]:
ovr6 = rocauc_ovr(y_sel_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.969664694280079

In [96]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel_over)

In [97]:
dat7 = pd.DataFrame(X_sel_test_over[:,-1])
dat7['test'] = y_sel_test_over

In [98]:
dat7

,0,test
0,NRS218,1
1,NRS260,1
2,NRS162,0
3,NRS177,0
4,NRS209,2
...,...,...
190,NRS383,1
191,NRS218,1
192,NRS209,2
193,SR2852,1


In [99]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [100]:
model2_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [110]:
model2_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 430us/step - loss: 1.2912 - accuracy: 0.4592 - val_loss: 0.8441 - val_accuracy: 0.5333
Epoch 2/1000
453/453 [==============================] - 0s 222us/step - loss: 0.7507 - accuracy: 0.6932 - val_loss: 0.6594 - val_accuracy: 0.7487
Epoch 3/1000
453/453 [==============================] - 0s 252us/step - loss: 0.6182 - accuracy: 0.7263 - val_loss: 0.5640 - val_accuracy: 0.7436
Epoch 4/1000
453/453 [==============================] - 0s 174us/step - loss: 0.5431 - accuracy: 0.7550 - val_loss: 0.5057 - val_accuracy: 0.8103
Epoch 5/1000
453/453 [==============================] - 0s 156us/step - loss: 0.4939 - accuracy: 0.8057 - val_loss: 0.4609 - val_accuracy: 0.8103
Epoch 6/1000
453/453 [==============================] - 0s 149us/step - loss: 0.4604 - accuracy: 0.7881 - val_loss: 0.4371 - val_accuracy: 0.8205
Epoch 7/1000
453/453 [==============================] - 0s 309us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 225us/step - loss: 0.2598 - accuracy: 0.8720 - val_loss: 0.3930 - val_accuracy: 0.8410
Epoch 113/1000
453/453 [==============================] - 0s 214us/step - loss: 0.2580 - accuracy: 0.8786 - val_loss: 0.3772 - val_accuracy: 0.8359
Epoch 114/1000
453/453 [==============================] - 0s 125us/step - loss: 0.2576 - accuracy: 0.8675 - val_loss: 0.3883 - val_accuracy: 0.8410
Epoch 115/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2588 - accuracy: 0.8720 - val_loss: 0.3999 - val_accuracy: 0.8308
Epoch 116/1000
453/453 [==============================] - 0s 239us/step - loss: 0.2630 - accuracy: 0.8742 - val_loss: 0.3788 - val_accuracy: 0.8359
Epoch 117/1000
453/453 [==============================] - 0s 244us/step - loss: 0.2653 - accuracy: 0.8720 - val_loss: 0.3932 - val_accuracy: 0.8359
Epoch 118/1000
453/453 [==============================] - 0s 166us/step - loss: 0.2583 - accuracy: 0.8631 - val_

453/453 [==============================] - 0s 137us/step - loss: 0.2403 - accuracy: 0.8852 - val_loss: 0.4434 - val_accuracy: 0.8359
Epoch 223/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2463 - accuracy: 0.8764 - val_loss: 0.4490 - val_accuracy: 0.8359
Epoch 224/1000
453/453 [==============================] - 0s 126us/step - loss: 0.2478 - accuracy: 0.8852 - val_loss: 0.4786 - val_accuracy: 0.8359
Epoch 225/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2460 - accuracy: 0.8764 - val_loss: 0.4543 - val_accuracy: 0.8359
Epoch 226/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2427 - accuracy: 0.8720 - val_loss: 0.4455 - val_accuracy: 0.8359
Epoch 227/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2435 - accuracy: 0.8786 - val_loss: 0.4585 - val_accuracy: 0.8410
Epoch 228/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2410 - accuracy: 0.8786 - val_loss: 0.4553 - 

453/453 [==============================] - 0s 108us/step - loss: 0.2412 - accuracy: 0.8764 - val_loss: 0.5032 - val_accuracy: 0.8410
Epoch 333/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2381 - accuracy: 0.8808 - val_loss: 0.4989 - val_accuracy: 0.8410
Epoch 334/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2359 - accuracy: 0.8852 - val_loss: 0.4988 - val_accuracy: 0.8410
Epoch 335/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2381 - accuracy: 0.8786 - val_loss: 0.5066 - val_accuracy: 0.8462
Epoch 336/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2371 - accuracy: 0.8808 - val_loss: 0.5006 - val_accuracy: 0.8410
Epoch 337/1000
453/453 [==============================] - 0s 140us/step - loss: 0.2391 - accuracy: 0.8764 - val_loss: 0.4927 - val_accuracy: 0.8410
Epoch 338/1000
453/453 [==============================] - 0s 140us/step - loss: 0.2344 - accuracy: 0.8830 - val_loss: 0.4981 - 

453/453 [==============================] - 0s 358us/step - loss: 0.2423 - accuracy: 0.8675 - val_loss: 0.5263 - val_accuracy: 0.8410
Epoch 443/1000
453/453 [==============================] - 0s 316us/step - loss: 0.2375 - accuracy: 0.8808 - val_loss: 0.5187 - val_accuracy: 0.8462
Epoch 444/1000
453/453 [==============================] - 0s 661us/step - loss: 0.2329 - accuracy: 0.8874 - val_loss: 0.5305 - val_accuracy: 0.8462
Epoch 445/1000
453/453 [==============================] - 0s 252us/step - loss: 0.2400 - accuracy: 0.8742 - val_loss: 0.5316 - val_accuracy: 0.8410
Epoch 446/1000
453/453 [==============================] - 0s 303us/step - loss: 0.2328 - accuracy: 0.8874 - val_loss: 0.5324 - val_accuracy: 0.8462
Epoch 447/1000
453/453 [==============================] - 0s 382us/step - loss: 0.2342 - accuracy: 0.8874 - val_loss: 0.5334 - val_accuracy: 0.8462
Epoch 448/1000
453/453 [==============================] - 0s 564us/step - loss: 0.2348 - accuracy: 0.8852 - val_loss: 0.5365 - 

453/453 [==============================] - 0s 181us/step - loss: 0.2335 - accuracy: 0.8874 - val_loss: 0.5994 - val_accuracy: 0.8410
Epoch 553/1000
453/453 [==============================] - 0s 153us/step - loss: 0.2341 - accuracy: 0.8874 - val_loss: 0.6061 - val_accuracy: 0.8410
Epoch 554/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2392 - accuracy: 0.8808 - val_loss: 0.5977 - val_accuracy: 0.8410
Epoch 555/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2311 - accuracy: 0.8874 - val_loss: 0.5982 - val_accuracy: 0.8462
Epoch 556/1000
453/453 [==============================] - 0s 162us/step - loss: 0.2331 - accuracy: 0.8896 - val_loss: 0.6090 - val_accuracy: 0.8462
Epoch 557/1000
453/453 [==============================] - 0s 161us/step - loss: 0.2393 - accuracy: 0.8786 - val_loss: 0.6047 - val_accuracy: 0.8462
Epoch 558/1000
453/453 [==============================] - 0s 173us/step - loss: 0.2347 - accuracy: 0.8742 - val_loss: 0.5936 - 

453/453 [==============================] - 0s 135us/step - loss: 0.2309 - accuracy: 0.8874 - val_loss: 0.6489 - val_accuracy: 0.8410
Epoch 663/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2308 - accuracy: 0.8874 - val_loss: 0.6258 - val_accuracy: 0.8410
Epoch 664/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2324 - accuracy: 0.8786 - val_loss: 0.6289 - val_accuracy: 0.8410
Epoch 665/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2335 - accuracy: 0.8808 - val_loss: 0.6453 - val_accuracy: 0.8462
Epoch 666/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2347 - accuracy: 0.8852 - val_loss: 0.6521 - val_accuracy: 0.8410
Epoch 667/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2328 - accuracy: 0.8852 - val_loss: 0.6539 - val_accuracy: 0.8410
Epoch 668/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2284 - accuracy: 0.8874 - val_loss: 0.6601 - 

453/453 [==============================] - 0s 272us/step - loss: 0.2394 - accuracy: 0.8808 - val_loss: 0.7048 - val_accuracy: 0.8513
Epoch 773/1000
453/453 [==============================] - 0s 272us/step - loss: 0.2333 - accuracy: 0.8852 - val_loss: 0.6935 - val_accuracy: 0.8513
Epoch 774/1000
453/453 [==============================] - 0s 207us/step - loss: 0.2316 - accuracy: 0.8742 - val_loss: 0.7032 - val_accuracy: 0.8462
Epoch 775/1000
453/453 [==============================] - 0s 245us/step - loss: 0.2363 - accuracy: 0.8852 - val_loss: 0.6969 - val_accuracy: 0.8513
Epoch 776/1000
453/453 [==============================] - 0s 179us/step - loss: 0.2346 - accuracy: 0.8896 - val_loss: 0.7090 - val_accuracy: 0.8462
Epoch 777/1000
453/453 [==============================] - 0s 332us/step - loss: 0.2367 - accuracy: 0.8764 - val_loss: 0.7032 - val_accuracy: 0.8462
Epoch 778/1000
453/453 [==============================] - 0s 292us/step - loss: 0.2279 - accuracy: 0.8874 - val_loss: 0.7040 - 

453/453 [==============================] - 0s 112us/step - loss: 0.2367 - accuracy: 0.8786 - val_loss: 0.7566 - val_accuracy: 0.8462
Epoch 883/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2325 - accuracy: 0.8852 - val_loss: 0.7586 - val_accuracy: 0.8462
Epoch 884/1000
453/453 [==============================] - 0s 143us/step - loss: 0.2292 - accuracy: 0.8874 - val_loss: 0.7597 - val_accuracy: 0.8462
Epoch 885/1000
453/453 [==============================] - 0s 162us/step - loss: 0.2301 - accuracy: 0.8874 - val_loss: 0.7594 - val_accuracy: 0.8462
Epoch 886/1000
453/453 [==============================] - 0s 157us/step - loss: 0.2323 - accuracy: 0.8874 - val_loss: 0.7604 - val_accuracy: 0.8462
Epoch 887/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2291 - accuracy: 0.8874 - val_loss: 0.7647 - val_accuracy: 0.8462
Epoch 888/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2298 - accuracy: 0.8874 - val_loss: 0.7621 - 

453/453 [==============================] - 0s 305us/step - loss: 0.2304 - accuracy: 0.8786 - val_loss: 0.8109 - val_accuracy: 0.8462
Epoch 993/1000
453/453 [==============================] - 0s 226us/step - loss: 0.2297 - accuracy: 0.8874 - val_loss: 0.7648 - val_accuracy: 0.8462
Epoch 994/1000
453/453 [==============================] - 0s 212us/step - loss: 0.2303 - accuracy: 0.8830 - val_loss: 0.7886 - val_accuracy: 0.8462
Epoch 995/1000
453/453 [==============================] - 0s 249us/step - loss: 0.2286 - accuracy: 0.8874 - val_loss: 0.8019 - val_accuracy: 0.8462
Epoch 996/1000
453/453 [==============================] - 0s 318us/step - loss: 0.2287 - accuracy: 0.8852 - val_loss: 0.8095 - val_accuracy: 0.8462
Epoch 997/1000
453/453 [==============================] - 0s 334us/step - loss: 0.2301 - accuracy: 0.8852 - val_loss: 0.8219 - val_accuracy: 0.8513
Epoch 998/1000
453/453 [==============================] - 0s 328us/step - loss: 0.2290 - accuracy: 0.8874 - val_loss: 0.8138 - 

In [170]:
acc_test2_over3 = model2_over3.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over3*100))

195/195 [==============================] - 0s 58us/step
over-sampling test accuracy: 84.62%


In [112]:
pred7 = model2_over3.predict_classes(X_sel_test_over)
pred7

array([1, 1, 0, 0, 2, 0, 0, 0, 2, 2, 1, 2, 0, 1, 0, 1, 0, 0, 0, 1, 2, 2,
       2, 2, 1, 1, 0, 0, 1, 0, 1, 0, 0, 2, 2, 1, 0, 0, 2, 0, 2, 0, 2, 0,
       1, 0, 1, 2, 1, 2, 2, 0, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 2, 2, 1, 1,
       2, 0, 0, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 1,
       0, 2, 0, 2, 2, 0, 2, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0, 0, 2, 1, 0, 1,
       0, 1, 2, 1, 2, 1, 0, 0, 0, 1, 0, 1, 2, 1, 0, 2, 0, 0, 0, 2, 1, 0,
       2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 2, 1, 0, 2, 1, 1, 1, 2, 2, 2, 0, 0,
       2, 1, 2, 2, 1, 2, 2, 1, 1, 0, 2, 0, 0, 2, 1, 1, 0, 0, 0, 0, 2, 1,
       1, 0, 1, 1, 2, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 1, 2, 1, 0])

In [113]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS218,1,1
1,NRS260,1,1
2,NRS162,0,0
3,NRS177,0,0
4,NRS209,2,2
...,...,...,...
190,NRS383,1,0
191,NRS218,1,1
192,NRS209,2,2
193,SR2852,1,1


In [114]:
proba7 = model2_over3.predict_proba(X_sel_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [115]:
dat_proba7

,0,1,2
0,6.953657e-05,9.999305e-01,3.132419e-10
1,7.616657e-09,1.000000e+00,8.662086e-13
2,8.434604e-01,1.565396e-01,4.034365e-09
3,9.995130e-01,4.869479e-04,7.161992e-10
4,2.713214e-09,6.656316e-09,1.000000e+00
...,...,...,...
190,5.477194e-01,4.522807e-01,1.761375e-08
191,6.953657e-05,9.999305e-01,3.132419e-10
192,2.713214e-09,6.656316e-09,1.000000e+00
193,9.956684e-12,1.000000e+00,7.441288e-26


In [116]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [117]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p17s.csv", index = False,
         header=None)

In [174]:
hist2_over3 = model2_over3.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 283us/step - loss: 0.2323 - accuracy: 0.8786 - val_loss: 0.7526 - val_accuracy: 0.8513
Epoch 2/1000
453/453 [==============================] - 0s 181us/step - loss: 0.2344 - accuracy: 0.8830 - val_loss: 0.7470 - val_accuracy: 0.8513
Epoch 3/1000
453/453 [==============================] - 0s 222us/step - loss: 0.2391 - accuracy: 0.8830 - val_loss: 0.7531 - val_accuracy: 0.8513
Epoch 4/1000
453/453 [==============================] - 0s 203us/step - loss: 0.2308 - accuracy: 0.8830 - val_loss: 0.7586 - val_accuracy: 0.8513
Epoch 5/1000
453/453 [==============================] - 0s 169us/step - loss: 0.2312 - accuracy: 0.8808 - val_loss: 0.7515 - val_accuracy: 0.8513
Epoch 6/1000
453/453 [==============================] - 0s 177us/step - loss: 0.2316 - accuracy: 0.8874 - val_loss: 0.7487 - val_accuracy: 0.8513
Epoch 7/1000
453/453 [==============================] - 0s 212us/step - loss: 

453/453 [==============================] - 0s 115us/step - loss: 0.2275 - accuracy: 0.8874 - val_loss: 0.8265 - val_accuracy: 0.8513
Epoch 113/1000
453/453 [==============================] - 0s 113us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 0.8288 - val_accuracy: 0.8462
Epoch 114/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2279 - accuracy: 0.8852 - val_loss: 0.8322 - val_accuracy: 0.8513
Epoch 115/1000
453/453 [==============================] - 0s 175us/step - loss: 0.2307 - accuracy: 0.8852 - val_loss: 0.8268 - val_accuracy: 0.8513
Epoch 116/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2297 - accuracy: 0.8874 - val_loss: 0.8284 - val_accuracy: 0.8462
Epoch 117/1000
453/453 [==============================] - 0s 116us/step - loss: 0.2290 - accuracy: 0.8874 - val_loss: 0.8274 - val_accuracy: 0.8462
Epoch 118/1000
453/453 [==============================] - 0s 92us/step - loss: 0.2298 - accuracy: 0.8852 - val_loss: 0.8339 - v

453/453 [==============================] - 0s 170us/step - loss: 0.2279 - accuracy: 0.8852 - val_loss: 0.8430 - val_accuracy: 0.8667
Epoch 223/1000
453/453 [==============================] - 0s 195us/step - loss: 0.2289 - accuracy: 0.8874 - val_loss: 0.8474 - val_accuracy: 0.8667
Epoch 224/1000
453/453 [==============================] - 0s 225us/step - loss: 0.2295 - accuracy: 0.8874 - val_loss: 0.8434 - val_accuracy: 0.8667
Epoch 225/1000
453/453 [==============================] - 0s 183us/step - loss: 0.2304 - accuracy: 0.8874 - val_loss: 0.8449 - val_accuracy: 0.8667
Epoch 226/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2273 - accuracy: 0.8874 - val_loss: 0.8466 - val_accuracy: 0.8667
Epoch 227/1000
453/453 [==============================] - 0s 152us/step - loss: 0.2288 - accuracy: 0.8874 - val_loss: 0.8478 - val_accuracy: 0.8667
Epoch 228/1000
453/453 [==============================] - 0s 168us/step - loss: 0.2298 - accuracy: 0.8874 - val_loss: 0.8521 - 

453/453 [==============================] - 0s 150us/step - loss: 0.2283 - accuracy: 0.8874 - val_loss: 0.8446 - val_accuracy: 0.8462
Epoch 333/1000
453/453 [==============================] - 0s 140us/step - loss: 0.2282 - accuracy: 0.8852 - val_loss: 0.8496 - val_accuracy: 0.8462
Epoch 334/1000
453/453 [==============================] - 0s 135us/step - loss: 0.2290 - accuracy: 0.8874 - val_loss: 0.8461 - val_accuracy: 0.8462
Epoch 335/1000
453/453 [==============================] - 0s 137us/step - loss: 0.2285 - accuracy: 0.8852 - val_loss: 0.8457 - val_accuracy: 0.8462
Epoch 336/1000
453/453 [==============================] - 0s 134us/step - loss: 0.2340 - accuracy: 0.8874 - val_loss: 0.8516 - val_accuracy: 0.8462
Epoch 337/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2351 - accuracy: 0.8808 - val_loss: 0.8531 - val_accuracy: 0.8462
Epoch 338/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2292 - accuracy: 0.8874 - val_loss: 0.8485 - 

453/453 [==============================] - 0s 126us/step - loss: 0.2313 - accuracy: 0.8808 - val_loss: 0.9070 - val_accuracy: 0.8462
Epoch 443/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2297 - accuracy: 0.8808 - val_loss: 0.9025 - val_accuracy: 0.8462
Epoch 444/1000
453/453 [==============================] - 0s 141us/step - loss: 0.2314 - accuracy: 0.8874 - val_loss: 0.8991 - val_accuracy: 0.8462
Epoch 445/1000
453/453 [==============================] - 0s 128us/step - loss: 0.2297 - accuracy: 0.8874 - val_loss: 0.9065 - val_accuracy: 0.8462
Epoch 446/1000
453/453 [==============================] - 0s 104us/step - loss: 0.2315 - accuracy: 0.8830 - val_loss: 0.8951 - val_accuracy: 0.8462
Epoch 447/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2280 - accuracy: 0.8830 - val_loss: 0.8964 - val_accuracy: 0.8462
Epoch 448/1000
453/453 [==============================] - 0s 99us/step - loss: 0.2277 - accuracy: 0.8852 - val_loss: 0.9032 - v

453/453 [==============================] - 0s 282us/step - loss: 0.2278 - accuracy: 0.8874 - val_loss: 0.9286 - val_accuracy: 0.8462
Epoch 553/1000
453/453 [==============================] - 0s 155us/step - loss: 0.2298 - accuracy: 0.8874 - val_loss: 0.9265 - val_accuracy: 0.8462
Epoch 554/1000
453/453 [==============================] - 0s 138us/step - loss: 0.2280 - accuracy: 0.8874 - val_loss: 0.9286 - val_accuracy: 0.8462
Epoch 555/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2283 - accuracy: 0.8830 - val_loss: 0.9303 - val_accuracy: 0.8462
Epoch 556/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2271 - accuracy: 0.8852 - val_loss: 0.9328 - val_accuracy: 0.8462
Epoch 557/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2258 - accuracy: 0.8874 - val_loss: 0.9361 - val_accuracy: 0.8462
Epoch 558/1000
453/453 [==============================] - 0s 115us/step - loss: 0.2279 - accuracy: 0.8874 - val_loss: 0.9356 - 

453/453 [==============================] - 0s 125us/step - loss: 0.2295 - accuracy: 0.8808 - val_loss: 0.9318 - val_accuracy: 0.8564
Epoch 663/1000
453/453 [==============================] - 0s 228us/step - loss: 0.2324 - accuracy: 0.8874 - val_loss: 0.9360 - val_accuracy: 0.8615
Epoch 664/1000
453/453 [==============================] - 0s 177us/step - loss: 0.2292 - accuracy: 0.8874 - val_loss: 0.9373 - val_accuracy: 0.8615
Epoch 665/1000
453/453 [==============================] - 0s 205us/step - loss: 0.2289 - accuracy: 0.8874 - val_loss: 0.9323 - val_accuracy: 0.8564
Epoch 666/1000
453/453 [==============================] - 0s 194us/step - loss: 0.2284 - accuracy: 0.8852 - val_loss: 0.9351 - val_accuracy: 0.8564
Epoch 667/1000
453/453 [==============================] - 0s 198us/step - loss: 0.2276 - accuracy: 0.8874 - val_loss: 0.9404 - val_accuracy: 0.8564
Epoch 668/1000
453/453 [==============================] - 0s 185us/step - loss: 0.2311 - accuracy: 0.8874 - val_loss: 0.9412 - 

453/453 [==============================] - 0s 130us/step - loss: 0.2286 - accuracy: 0.8874 - val_loss: 0.9553 - val_accuracy: 0.8615
Epoch 773/1000
453/453 [==============================] - 0s 194us/step - loss: 0.2314 - accuracy: 0.8874 - val_loss: 0.9607 - val_accuracy: 0.8615
Epoch 774/1000
453/453 [==============================] - 0s 193us/step - loss: 0.2273 - accuracy: 0.8874 - val_loss: 0.9518 - val_accuracy: 0.8615
Epoch 775/1000
453/453 [==============================] - 0s 239us/step - loss: 0.2303 - accuracy: 0.8896 - val_loss: 0.9612 - val_accuracy: 0.8615
Epoch 776/1000
453/453 [==============================] - 0s 219us/step - loss: 0.2314 - accuracy: 0.8874 - val_loss: 0.9615 - val_accuracy: 0.8615
Epoch 777/1000
453/453 [==============================] - 0s 271us/step - loss: 0.2275 - accuracy: 0.8874 - val_loss: 0.9605 - val_accuracy: 0.8615
Epoch 778/1000
453/453 [==============================] - 0s 247us/step - loss: 0.2285 - accuracy: 0.8764 - val_loss: 0.9596 - 

453/453 [==============================] - 0s 126us/step - loss: 0.2279 - accuracy: 0.8764 - val_loss: 0.9585 - val_accuracy: 0.8769
Epoch 883/1000
453/453 [==============================] - 0s 129us/step - loss: 0.2291 - accuracy: 0.8852 - val_loss: 0.9629 - val_accuracy: 0.8718
Epoch 884/1000
453/453 [==============================] - 0s 134us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 0.9652 - val_accuracy: 0.8718
Epoch 885/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 0.9622 - val_accuracy: 0.8718
Epoch 886/1000
453/453 [==============================] - 0s 118us/step - loss: 0.2286 - accuracy: 0.8874 - val_loss: 0.9661 - val_accuracy: 0.8718
Epoch 887/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2278 - accuracy: 0.8808 - val_loss: 0.9674 - val_accuracy: 0.8718
Epoch 888/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2281 - accuracy: 0.8874 - val_loss: 0.9653 - 

453/453 [==============================] - 0s 110us/step - loss: 0.2273 - accuracy: 0.8874 - val_loss: 1.0261 - val_accuracy: 0.8667
Epoch 993/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2287 - accuracy: 0.8830 - val_loss: 1.0257 - val_accuracy: 0.8667
Epoch 994/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2299 - accuracy: 0.8874 - val_loss: 1.0287 - val_accuracy: 0.8667
Epoch 995/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2281 - accuracy: 0.8874 - val_loss: 1.0289 - val_accuracy: 0.8667
Epoch 996/1000
453/453 [==============================] - 0s 114us/step - loss: 0.2282 - accuracy: 0.8874 - val_loss: 1.0259 - val_accuracy: 0.8667
Epoch 997/1000
453/453 [==============================] - 0s 110us/step - loss: 0.2288 - accuracy: 0.8874 - val_loss: 1.0284 - val_accuracy: 0.8667
Epoch 998/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2298 - accuracy: 0.8830 - val_loss: 1.0312 - 

In [175]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over3.history['accuracy'])*100))

over-sampling train accuracy: 88.55%


In [101]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [102]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS210,0,0,6.132076e-01,2.812180e-01,1.055744e-01
1,p0006kpresabs_qual,NRS205,2,2,1.993202e-04,6.834937e-07,9.998000e-01
2,p0006kpresabs_qual,312,2,1,3.589463e-01,3.982787e-01,2.427750e-01
3,p0006kpresabs_qual,GA15,2,1,3.589463e-01,3.982787e-01,2.427750e-01
4,p0006kpresabs_qual,SR4035,0,1,3.589463e-01,3.982787e-01,2.427750e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,NRS383,1,0,5.477194e-01,4.522807e-01,1.761374e-08
985,p0017Skpresabs_qual,NRS218,1,1,6.953657e-05,9.999305e-01,3.132419e-10
986,p0017Skpresabs_qual,NRS209,2,2,2.713214e-09,6.656316e-09,1.000000e+00
987,p0017Skpresabs_qual,SR2852,1,1,9.956684e-12,1.000000e+00,7.441288e-26


In [103]:
y_prob7 = df_proba7[df_proba7['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[6.9536570e-05, 9.9993050e-01, 3.1324188e-10],
       [7.6166570e-09, 1.0000000e+00, 8.6620863e-13],
       [8.4346040e-01, 1.5653962e-01, 4.0343653e-09],
       [9.9951303e-01, 4.8694790e-04, 7.1619916e-10],
       [2.7132139e-09, 6.6563160e-09, 1.0000000e+00],
       [7.1770490e-01, 2.8229514e-01, 1.4094924e-08],
       [9.9999833e-01, 1.6947181e-06, 4.4169048e-12],
       [8.4346040e-01, 1.5653962e-01, 4.0343653e-09],
       [2.7132139e-09, 6.6563160e-09, 1.0000000e+00],
       [2.7132139e-09, 6.6563160e-09, 1.0000000e+00],
       [1.6357395e-01, 8.3642600e-01, 2.6581544e-09],
       [1.1604803e-08, 1.5658442e-08, 1.0000000e+00],
       [7.1770490e-01, 2.8229514e-01, 1.4094924e-08],
       [2.3267869e-11, 1.0000000e+00, 1.3901998e-10],
       [8.6024284e-01, 1.3975719e-01, 3.1456690e-09],
       [1.9058405e-17, 1.0000000e+00, 1.4747358e-27],
       [9.5952930e-01, 4.0470740e-02, 1.8783513e-08],
       [6.3022670e-01, 3.6977336e-01, 7.8539990e-09],
       [1.0000000e+00, 6.150

In [104]:
ovo7 = rocauc_ovo(y_sel_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9393688362919131

In [105]:
ovr7 = rocauc_ovr(y_sel_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9393688362919131

In [106]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel_over)

In [107]:
dat8 = pd.DataFrame(X_sel_test_over[:,-1])
dat8['test'] = y_sel_test_over

In [108]:
dat8

,0,test
0,NRS209,2
1,NRS255,1
2,NRS119,0
3,NRS071,0
4,NRS002,0
...,...,...
190,CFBRSa30,0
191,NRS383,1
192,NRS110,2
193,NRS209,2


In [109]:
X_sel_train_over = X_sel_train_over[:,:-1]
X_sel_test_over = X_sel_test_over[:,:-1]

In [122]:
model2_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [123]:
model2_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 295us/step - loss: 1.0143 - accuracy: 0.4393 - val_loss: 0.8758 - val_accuracy: 0.7231
Epoch 2/1000
453/453 [==============================] - 0s 114us/step - loss: 0.8130 - accuracy: 0.7108 - val_loss: 0.7375 - val_accuracy: 0.6923
Epoch 3/1000
453/453 [==============================] - 0s 113us/step - loss: 0.6968 - accuracy: 0.7263 - val_loss: 0.6457 - val_accuracy: 0.7231
Epoch 4/1000
453/453 [==============================] - 0s 111us/step - loss: 0.6178 - accuracy: 0.7395 - val_loss: 0.5821 - val_accuracy: 0.7744
Epoch 5/1000
453/453 [==============================] - 0s 138us/step - loss: 0.5638 - accuracy: 0.7881 - val_loss: 0.5373 - val_accuracy: 0.7795
Epoch 6/1000
453/453 [==============================] - 0s 195us/step - loss: 0.5246 - accuracy: 0.7969 - val_loss: 0.5048 - val_accuracy: 0.7641
Epoch 7/1000
453/453 [==============================] - 0s 133us/step - loss: 

Epoch 112/1000
453/453 [==============================] - 0s 62us/step - loss: 0.2861 - accuracy: 0.8521 - val_loss: 0.3144 - val_accuracy: 0.8513
Epoch 113/1000
453/453 [==============================] - 0s 61us/step - loss: 0.2846 - accuracy: 0.8631 - val_loss: 0.3117 - val_accuracy: 0.8308
Epoch 114/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2831 - accuracy: 0.8499 - val_loss: 0.3092 - val_accuracy: 0.8256
Epoch 115/1000
453/453 [==============================] - 0s 59us/step - loss: 0.2825 - accuracy: 0.8587 - val_loss: 0.3096 - val_accuracy: 0.8256
Epoch 116/1000
453/453 [==============================] - 0s 62us/step - loss: 0.2812 - accuracy: 0.8653 - val_loss: 0.3101 - val_accuracy: 0.8256
Epoch 117/1000
453/453 [==============================] - 0s 58us/step - loss: 0.2798 - accuracy: 0.8675 - val_loss: 0.3094 - val_accuracy: 0.8308
Epoch 118/1000
453/453 [==============================] - 0s 61us/step - loss: 0.2813 - accuracy: 0.8587 - val_loss: 0

Epoch 168/1000
453/453 [==============================] - 0s 60us/step - loss: 0.2704 - accuracy: 0.8675 - val_loss: 0.2971 - val_accuracy: 0.8667
Epoch 169/1000
453/453 [==============================] - 0s 76us/step - loss: 0.2699 - accuracy: 0.8653 - val_loss: 0.2950 - val_accuracy: 0.8256
Epoch 170/1000
453/453 [==============================] - 0s 62us/step - loss: 0.2800 - accuracy: 0.8587 - val_loss: 0.3013 - val_accuracy: 0.8615
Epoch 171/1000
453/453 [==============================] - 0s 59us/step - loss: 0.2663 - accuracy: 0.8653 - val_loss: 0.2954 - val_accuracy: 0.8308
Epoch 172/1000
453/453 [==============================] - 0s 60us/step - loss: 0.2726 - accuracy: 0.8631 - val_loss: 0.2984 - val_accuracy: 0.8615
Epoch 173/1000
453/453 [==============================] - 0s 58us/step - loss: 0.2663 - accuracy: 0.8742 - val_loss: 0.2950 - val_accuracy: 0.8615
Epoch 174/1000
453/453 [==============================] - 0s 57us/step - loss: 0.2717 - accuracy: 0.8609 - val_loss: 0

Epoch 224/1000
453/453 [==============================] - 0s 82us/step - loss: 0.2636 - accuracy: 0.8698 - val_loss: 0.2911 - val_accuracy: 0.8615
Epoch 225/1000
453/453 [==============================] - 0s 149us/step - loss: 0.2623 - accuracy: 0.8698 - val_loss: 0.2870 - val_accuracy: 0.8615
Epoch 226/1000
453/453 [==============================] - 0s 130us/step - loss: 0.2570 - accuracy: 0.8764 - val_loss: 0.2868 - val_accuracy: 0.8615
Epoch 227/1000
453/453 [==============================] - 0s 105us/step - loss: 0.2579 - accuracy: 0.8786 - val_loss: 0.2895 - val_accuracy: 0.8513
Epoch 228/1000
453/453 [==============================] - 0s 109us/step - loss: 0.2554 - accuracy: 0.8764 - val_loss: 0.2892 - val_accuracy: 0.8615
Epoch 229/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2547 - accuracy: 0.8786 - val_loss: 0.2872 - val_accuracy: 0.8615
Epoch 230/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2629 - accuracy: 0.8742 - val_l

Epoch 335/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2495 - accuracy: 0.8786 - val_loss: 0.2769 - val_accuracy: 0.8769
Epoch 336/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2452 - accuracy: 0.8786 - val_loss: 0.2737 - val_accuracy: 0.8821
Epoch 337/1000
453/453 [==============================] - 0s 93us/step - loss: 0.2457 - accuracy: 0.8808 - val_loss: 0.2736 - val_accuracy: 0.8769
Epoch 338/1000
453/453 [==============================] - 0s 119us/step - loss: 0.2583 - accuracy: 0.8653 - val_loss: 0.2825 - val_accuracy: 0.8769
Epoch 339/1000
453/453 [==============================] - 0s 96us/step - loss: 0.2518 - accuracy: 0.8808 - val_loss: 0.2750 - val_accuracy: 0.8769
Epoch 340/1000
453/453 [==============================] - 0s 112us/step - loss: 0.2479 - accuracy: 0.8830 - val_loss: 0.2768 - val_accuracy: 0.8769
Epoch 341/1000
453/453 [==============================] - 0s 70us/step - loss: 0.2457 - accuracy: 0.8720 - val_loss

...

Epoch 722/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2344 - accuracy: 0.8852 - val_loss: 0.2691 - val_accuracy: 0.8718
Epoch 723/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2317 - accuracy: 0.8830 - val_loss: 0.2747 - val_accuracy: 0.8718
Epoch 724/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2347 - accuracy: 0.8830 - val_loss: 0.2705 - val_accuracy: 0.8718
Epoch 725/1000
453/453 [==============================] - 0s 92us/step - loss: 0.2325 - accuracy: 0.8852 - val_loss: 0.2740 - val_accuracy: 0.8718
Epoch 726/1000
453/453 [==============================] - 0s 86us/step - loss: 0.2336 - accuracy: 0.8808 - val_loss: 0.2771 - val_accuracy: 0.8718
Epoch 727/1000
453/453 [==============================] - 0s 79us/step - loss: 0.2342 - accuracy: 0.8830 - val_loss: 0.2684 - val_accuracy: 0.8718
Epoch 728/1000
453/453 [==============================] - 0s 75us/step - loss: 0.2373 - accuracy: 0.8808 - val_loss: 

Epoch 778/1000
453/453 [==============================] - 0s 68us/step - loss: 0.2380 - accuracy: 0.8808 - val_loss: 0.2721 - val_accuracy: 0.8821
Epoch 779/1000
453/453 [==============================] - 0s 92us/step - loss: 0.2375 - accuracy: 0.8830 - val_loss: 0.2686 - val_accuracy: 0.8821
Epoch 780/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2314 - accuracy: 0.8852 - val_loss: 0.2760 - val_accuracy: 0.8769
Epoch 781/1000
453/453 [==============================] - 0s 111us/step - loss: 0.2352 - accuracy: 0.8786 - val_loss: 0.2690 - val_accuracy: 0.8821
Epoch 782/1000
453/453 [==============================] - 0s 68us/step - loss: 0.2313 - accuracy: 0.8830 - val_loss: 0.2705 - val_accuracy: 0.8769
Epoch 783/1000
453/453 [==============================] - 0s 99us/step - loss: 0.2327 - accuracy: 0.8830 - val_loss: 0.2710 - val_accuracy: 0.8769
Epoch 784/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2393 - accuracy: 0.8852 - val_loss: 

Epoch 834/1000
453/453 [==============================] - 0s 75us/step - loss: 0.2333 - accuracy: 0.8786 - val_loss: 0.2737 - val_accuracy: 0.8769
Epoch 835/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2360 - accuracy: 0.8852 - val_loss: 0.2708 - val_accuracy: 0.8821
Epoch 836/1000
453/453 [==============================] - 0s 94us/step - loss: 0.2359 - accuracy: 0.8830 - val_loss: 0.2660 - val_accuracy: 0.8769
Epoch 837/1000
453/453 [==============================] - 0s 75us/step - loss: 0.2291 - accuracy: 0.8852 - val_loss: 0.2747 - val_accuracy: 0.8821
Epoch 838/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2337 - accuracy: 0.8852 - val_loss: 0.2740 - val_accuracy: 0.8821
Epoch 839/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2381 - accuracy: 0.8786 - val_loss: 0.2705 - val_accuracy: 0.8769
Epoch 840/1000
453/453 [==============================] - 0s 70us/step - loss: 0.2327 - accuracy: 0.8852 - val_loss: 0

Epoch 890/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2323 - accuracy: 0.8852 - val_loss: 0.2722 - val_accuracy: 0.8821
Epoch 891/1000
453/453 [==============================] - 0s 73us/step - loss: 0.2330 - accuracy: 0.8808 - val_loss: 0.2675 - val_accuracy: 0.8821
Epoch 892/1000
453/453 [==============================] - 0s 75us/step - loss: 0.2376 - accuracy: 0.8698 - val_loss: 0.2688 - val_accuracy: 0.8821
Epoch 893/1000
453/453 [==============================] - 0s 77us/step - loss: 0.2321 - accuracy: 0.8830 - val_loss: 0.2731 - val_accuracy: 0.8821
Epoch 894/1000
453/453 [==============================] - 0s 80us/step - loss: 0.2311 - accuracy: 0.8830 - val_loss: 0.2696 - val_accuracy: 0.8821
Epoch 895/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2306 - accuracy: 0.8852 - val_loss: 0.2693 - val_accuracy: 0.8821
Epoch 896/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2305 - accuracy: 0.8852 - val_loss: 0

Epoch 946/1000
453/453 [==============================] - 0s 81us/step - loss: 0.2297 - accuracy: 0.8852 - val_loss: 0.2665 - val_accuracy: 0.8821
Epoch 947/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2322 - accuracy: 0.8830 - val_loss: 0.2706 - val_accuracy: 0.8821
Epoch 948/1000
453/453 [==============================] - 0s 78us/step - loss: 0.2411 - accuracy: 0.8499 - val_loss: 0.2664 - val_accuracy: 0.8821
Epoch 949/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2379 - accuracy: 0.8720 - val_loss: 0.2653 - val_accuracy: 0.8821
Epoch 950/1000
453/453 [==============================] - 0s 70us/step - loss: 0.2320 - accuracy: 0.8808 - val_loss: 0.2717 - val_accuracy: 0.8821
Epoch 951/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2362 - accuracy: 0.8830 - val_loss: 0.2642 - val_accuracy: 0.8821
Epoch 952/1000
453/453 [==============================] - 0s 70us/step - loss: 0.2325 - accuracy: 0.8830 - val_loss: 0

In [191]:
acc_test2_over4 = model2_over4.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test2_over4*100))

195/195 [==============================] - 0s 58us/step
over-sampling test accuracy: 87.18%


In [125]:
pred8 = model2_over4.predict_classes(X_sel_test_over)
pred8

array([2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 2, 2, 0, 0, 1, 2,
       1, 0, 1, 0, 2, 1, 2, 0, 1, 1, 1, 0, 0, 1, 2, 1, 0, 2, 1, 1, 2, 0,
       2, 2, 0, 1, 2, 0, 1, 0, 0, 2, 1, 0, 2, 2, 1, 0, 1, 2, 0, 1, 2, 1,
       0, 0, 0, 2, 0, 0, 2, 0, 1, 1, 0, 0, 2, 0, 1, 2, 2, 2, 1, 0, 1, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 0, 0, 1, 1, 0, 0, 2, 2, 2, 1, 0,
       1, 2, 1, 0, 0, 2, 1, 0, 2, 0, 0, 1, 2, 0, 1, 2, 1, 0, 0, 0, 0, 2,
       2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 2, 2, 0, 0, 2, 0, 1, 2, 2, 0, 2, 1,
       0, 0, 0, 2, 1, 0, 0, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0, 0, 2, 1, 2, 2,
       1, 2, 1, 1, 0, 2, 1, 2, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 0])

In [126]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,NRS209,2,2
1,NRS255,1,1
2,NRS119,0,1
3,NRS071,0,0
4,NRS002,0,1
...,...,...,...
190,CFBRSa30,0,0
191,NRS383,1,0
192,NRS110,2,2
193,NRS209,2,2


In [127]:
proba8 = model2_over4.predict_proba(X_sel_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [128]:
dat_proba8

,0,1,2
0,3.604249e-12,2.698129e-07,9.999998e-01
1,9.804833e-07,9.999990e-01,1.326707e-12
2,4.475308e-01,5.524690e-01,2.029845e-07
3,6.129044e-01,3.870795e-01,1.601290e-05
4,4.475308e-01,5.524690e-01,2.029845e-07
...,...,...,...
190,7.207667e-01,2.792331e-01,2.571588e-07
191,6.129044e-01,3.870795e-01,1.601290e-05
192,3.260306e-07,7.910664e-07,9.999989e-01
193,3.604249e-12,2.698129e-07,9.999998e-01


In [129]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [130]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p17s.csv", index = False,
         header=None)

In [195]:
hist2_over4 = model2_over4.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 453 samples, validate on 195 samples
Epoch 1/1000
453/453 [==============================] - 0s 96us/step - loss: 0.2579 - accuracy: 0.8300 - val_loss: 0.2797 - val_accuracy: 0.8462
Epoch 2/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2644 - accuracy: 0.8698 - val_loss: 0.2789 - val_accuracy: 0.8718
Epoch 3/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2370 - accuracy: 0.8830 - val_loss: 0.2822 - val_accuracy: 0.8718
Epoch 4/1000
453/453 [==============================] - 0s 123us/step - loss: 0.2368 - accuracy: 0.8830 - val_loss: 0.2804 - val_accuracy: 0.8718
Epoch 5/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2441 - accuracy: 0.8764 - val_loss: 0.2771 - val_accuracy: 0.8718
Epoch 6/1000
453/453 [==============================] - 0s 108us/step - loss: 0.2392 - accuracy: 0.8830 - val_loss: 0.2789 - val_accuracy: 0.8718
Epoch 7/1000
453/453 [==============================] - 0s 119us/step - loss: 0

Epoch 57/1000
453/453 [==============================] - 0s 89us/step - loss: 0.2454 - accuracy: 0.8830 - val_loss: 0.2799 - val_accuracy: 0.8718
Epoch 58/1000
453/453 [==============================] - 0s 101us/step - loss: 0.2362 - accuracy: 0.8830 - val_loss: 0.2774 - val_accuracy: 0.8718
Epoch 59/1000
453/453 [==============================] - 0s 102us/step - loss: 0.2381 - accuracy: 0.8830 - val_loss: 0.2790 - val_accuracy: 0.8718
Epoch 60/1000
453/453 [==============================] - 0s 103us/step - loss: 0.2374 - accuracy: 0.8830 - val_loss: 0.2825 - val_accuracy: 0.8718
Epoch 61/1000
453/453 [==============================] - 0s 95us/step - loss: 0.2356 - accuracy: 0.8830 - val_loss: 0.2760 - val_accuracy: 0.8718
Epoch 62/1000
453/453 [==============================] - 0s 106us/step - loss: 0.2463 - accuracy: 0.8764 - val_loss: 0.2785 - val_accuracy: 0.8718
Epoch 63/1000
453/453 [==============================] - 0s 77us/step - loss: 0.2375 - accuracy: 0.8830 - val_loss: 0.27

Epoch 113/1000
453/453 [==============================] - 0s 226us/step - loss: 0.2345 - accuracy: 0.8808 - val_loss: 0.2807 - val_accuracy: 0.8718
Epoch 114/1000
453/453 [==============================] - 0s 248us/step - loss: 0.2358 - accuracy: 0.8830 - val_loss: 0.2801 - val_accuracy: 0.8718
Epoch 115/1000
453/453 [==============================] - 0s 201us/step - loss: 0.2381 - accuracy: 0.8830 - val_loss: 0.2819 - val_accuracy: 0.8718
Epoch 116/1000
453/453 [==============================] - 0s 596us/step - loss: 0.2419 - accuracy: 0.8830 - val_loss: 0.2869 - val_accuracy: 0.8718
Epoch 117/1000
453/453 [==============================] - 0s 160us/step - loss: 0.2394 - accuracy: 0.8808 - val_loss: 0.2787 - val_accuracy: 0.8718
Epoch 118/1000
453/453 [==============================] - 0s 210us/step - loss: 0.2383 - accuracy: 0.8830 - val_loss: 0.2809 - val_accuracy: 0.8718
Epoch 119/1000
453/453 [==============================] - 0s 127us/step - loss: 0.2357 - accuracy: 0.8830 - val_

453/453 [==============================] - 0s 124us/step - loss: 0.2340 - accuracy: 0.8830 - val_loss: 0.2824 - val_accuracy: 0.8718
Epoch 224/1000
453/453 [==============================] - 0s 124us/step - loss: 0.2363 - accuracy: 0.8830 - val_loss: 0.2826 - val_accuracy: 0.8718
Epoch 225/1000
453/453 [==============================] - 0s 97us/step - loss: 0.2366 - accuracy: 0.8830 - val_loss: 0.2796 - val_accuracy: 0.8718
Epoch 226/1000
453/453 [==============================] - 0s 121us/step - loss: 0.2349 - accuracy: 0.8830 - val_loss: 0.2810 - val_accuracy: 0.8718
Epoch 227/1000
453/453 [==============================] - 0s 117us/step - loss: 0.2354 - accuracy: 0.8830 - val_loss: 0.2885 - val_accuracy: 0.8718
Epoch 228/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2378 - accuracy: 0.8830 - val_loss: 0.2846 - val_accuracy: 0.8718
Epoch 229/1000
453/453 [==============================] - 0s 193us/step - loss: 0.2330 - accuracy: 0.8830 - val_loss: 0.2832 - v

453/453 [==============================] - 0s 91us/step - loss: 0.2374 - accuracy: 0.8786 - val_loss: 0.2900 - val_accuracy: 0.8718
Epoch 334/1000
453/453 [==============================] - 0s 120us/step - loss: 0.2363 - accuracy: 0.8830 - val_loss: 0.2851 - val_accuracy: 0.8718
Epoch 335/1000
453/453 [==============================] - 0s 140us/step - loss: 0.2356 - accuracy: 0.8830 - val_loss: 0.2924 - val_accuracy: 0.8718
Epoch 336/1000
453/453 [==============================] - 0s 80us/step - loss: 0.2418 - accuracy: 0.8830 - val_loss: 0.2849 - val_accuracy: 0.8718
Epoch 337/1000
453/453 [==============================] - 0s 76us/step - loss: 0.2385 - accuracy: 0.8742 - val_loss: 0.2822 - val_accuracy: 0.8718
Epoch 338/1000
453/453 [==============================] - 0s 107us/step - loss: 0.2332 - accuracy: 0.8764 - val_loss: 0.2817 - val_accuracy: 0.8718
Epoch 339/1000
453/453 [==============================] - 0s 87us/step - loss: 0.2338 - accuracy: 0.8830 - val_loss: 0.2894 - val_

...

453/453 [==============================] - 0s 73us/step - loss: 0.2323 - accuracy: 0.8852 - val_loss: 0.2919 - val_accuracy: 0.8769
Epoch 774/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2314 - accuracy: 0.8852 - val_loss: 0.2974 - val_accuracy: 0.8769
Epoch 775/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2341 - accuracy: 0.8852 - val_loss: 0.2857 - val_accuracy: 0.8769
Epoch 776/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2341 - accuracy: 0.8852 - val_loss: 0.2904 - val_accuracy: 0.8769
Epoch 777/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2306 - accuracy: 0.8852 - val_loss: 0.2906 - val_accuracy: 0.8769
Epoch 778/1000
453/453 [==============================] - 0s 69us/step - loss: 0.2315 - accuracy: 0.8852 - val_loss: 0.2894 - val_accuracy: 0.8769
Epoch 779/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2321 - accuracy: 0.8808 - val_loss: 0.2895 - val_acc

Epoch 829/1000
453/453 [==============================] - 0s 79us/step - loss: 0.2318 - accuracy: 0.8786 - val_loss: 0.2861 - val_accuracy: 0.8769
Epoch 830/1000
453/453 [==============================] - 0s 77us/step - loss: 0.2359 - accuracy: 0.8852 - val_loss: 0.3011 - val_accuracy: 0.8769
Epoch 831/1000
453/453 [==============================] - 0s 73us/step - loss: 0.2303 - accuracy: 0.8852 - val_loss: 0.2860 - val_accuracy: 0.8769
Epoch 832/1000
453/453 [==============================] - 0s 73us/step - loss: 0.2307 - accuracy: 0.8852 - val_loss: 0.2891 - val_accuracy: 0.8769
Epoch 833/1000
453/453 [==============================] - 0s 73us/step - loss: 0.2389 - accuracy: 0.8852 - val_loss: 0.2922 - val_accuracy: 0.8769
Epoch 834/1000
453/453 [==============================] - 0s 70us/step - loss: 0.2354 - accuracy: 0.8808 - val_loss: 0.2819 - val_accuracy: 0.8769
Epoch 835/1000
453/453 [==============================] - 0s 69us/step - loss: 0.2332 - accuracy: 0.8852 - val_loss: 0

Epoch 885/1000
453/453 [==============================] - 0s 72us/step - loss: 0.2308 - accuracy: 0.8852 - val_loss: 0.2917 - val_accuracy: 0.8769
Epoch 886/1000
453/453 [==============================] - 0s 69us/step - loss: 0.2336 - accuracy: 0.8786 - val_loss: 0.2843 - val_accuracy: 0.8769
Epoch 887/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2327 - accuracy: 0.8852 - val_loss: 0.2822 - val_accuracy: 0.8769
Epoch 888/1000
453/453 [==============================] - 0s 69us/step - loss: 0.2365 - accuracy: 0.8852 - val_loss: 0.2938 - val_accuracy: 0.8769
Epoch 889/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2337 - accuracy: 0.8852 - val_loss: 0.2863 - val_accuracy: 0.8769
Epoch 890/1000
453/453 [==============================] - 0s 81us/step - loss: 0.2392 - accuracy: 0.8852 - val_loss: 0.2820 - val_accuracy: 0.8769
Epoch 891/1000
453/453 [==============================] - 0s 75us/step - loss: 0.2323 - accuracy: 0.8830 - val_loss: 0

Epoch 941/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2298 - accuracy: 0.8852 - val_loss: 0.2778 - val_accuracy: 0.8769
Epoch 942/1000
453/453 [==============================] - 0s 75us/step - loss: 0.2299 - accuracy: 0.8808 - val_loss: 0.2843 - val_accuracy: 0.8769
Epoch 943/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2343 - accuracy: 0.8742 - val_loss: 0.2891 - val_accuracy: 0.8769
Epoch 944/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2323 - accuracy: 0.8852 - val_loss: 0.2814 - val_accuracy: 0.8769
Epoch 945/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2312 - accuracy: 0.8852 - val_loss: 0.2809 - val_accuracy: 0.8769
Epoch 946/1000
453/453 [==============================] - 0s 74us/step - loss: 0.2300 - accuracy: 0.8852 - val_loss: 0.2791 - val_accuracy: 0.8769
Epoch 947/1000
453/453 [==============================] - 0s 70us/step - loss: 0.2316 - accuracy: 0.8852 - val_loss: 0

Epoch 997/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2377 - accuracy: 0.8764 - val_loss: 0.2797 - val_accuracy: 0.8769
Epoch 998/1000
453/453 [==============================] - 0s 71us/step - loss: 0.2353 - accuracy: 0.8764 - val_loss: 0.2799 - val_accuracy: 0.8769
Epoch 999/1000
453/453 [==============================] - 0s 70us/step - loss: 0.2325 - accuracy: 0.8852 - val_loss: 0.2862 - val_accuracy: 0.8769
Epoch 1000/1000
453/453 [==============================] - 0s 73us/step - loss: 0.2304 - accuracy: 0.8830 - val_loss: 0.2823 - val_accuracy: 0.8615


In [196]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist2_over4.history['accuracy'])*100))

over-sampling train accuracy: 88.22%


In [110]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [111]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0006kpresabs_qual,NRS236,1,2,1.321970e-02,2.446264e-01,7.421539e-01
1,p0006kpresabs_qual,NRS113,2,2,3.478230e-02,2.806685e-01,6.845492e-01
2,p0006kpresabs_qual,CFBRSa23,0,0,4.090251e-01,3.405008e-01,2.504741e-01
3,p0006kpresabs_qual,NRS249,2,1,1.987907e-01,5.331044e-01,2.681049e-01
4,p0006kpresabs_qual,107,1,0,4.090251e-01,3.405008e-01,2.504741e-01
...,...,...,...,...,...,...,...
984,p0017Skpresabs_qual,CFBRSa30,0,0,7.207667e-01,2.792331e-01,2.571588e-07
985,p0017Skpresabs_qual,NRS383,1,0,6.129044e-01,3.870795e-01,1.601290e-05
986,p0017Skpresabs_qual,NRS110,2,2,3.260306e-07,7.910664e-07,9.999989e-01
987,p0017Skpresabs_qual,NRS209,2,2,3.604249e-12,2.698129e-07,9.999998e-01


In [112]:
y_prob8 = df_proba8[df_proba8['phage']=='p0017Skpresabs_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[3.60424880e-12, 2.69812900e-07, 9.99999760e-01],
       [9.80483300e-07, 9.99999050e-01, 1.32670730e-12],
       [4.47530840e-01, 5.52468960e-01, 2.02984500e-07],
       [6.12904400e-01, 3.87079540e-01, 1.60129040e-05],
       [4.47530840e-01, 5.52468960e-01, 2.02984500e-07],
       [7.20766660e-01, 2.79233100e-01, 2.57158800e-07],
       [7.20766660e-01, 2.79233100e-01, 2.57158800e-07],
       [7.20766660e-01, 2.79233100e-01, 2.57158800e-07],
       [2.43007960e-05, 9.99975700e-01, 1.72223550e-08],
       [7.20766660e-01, 2.79233100e-01, 2.57158800e-07],
       [6.55957400e-01, 3.44042360e-01, 2.02454000e-07],
       [7.62772740e-01, 2.37225900e-01, 1.38607190e-06],
       [3.60424880e-12, 2.69812900e-07, 9.99999760e-01],
       [8.57430100e-11, 1.00000000e+00, 5.02053550e-23],
       [6.12904400e-01, 3.87079540e-01, 1.60129040e-05],
       [6.55957400e-01, 3.44042360e-01, 2.02454000e-07],
       [3.26030600e-07, 7.91066400e-07, 9.99998900e-01],
       [3.26030600e-07, 7.91066

In [113]:
ovo8 = rocauc_ovo(y_sel_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9668639053254439

In [114]:
ovr8 = rocauc_ovr(y_sel_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9668639053254439

In [115]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9557495069033531

In [116]:
np.std(ovos2)

0.012848077634546077

In [117]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9557495069033531

In [118]:
np.std(ovrs2)

0.012848077634546077

In [197]:
accs_l_over = [acc_test2_over, acc_test2_over2, acc_test2_over3, acc_test2_over4]

In [198]:
mean_l_over = np.mean(accs_l_over)
print('over-sampling test accuracy mean after lasso: %.2f%%' % (mean_l_over*100))

over-sampling test accuracy mean after lasso: 85.77%


In [199]:
std_l_over = np.std(accs_l_over)
print('over-sampling test accuracy standard deviation after lasso:', std_l_over)

over-sampling test accuracy standard deviation after lasso: 0.01421863569918196


In [200]:
accs_train_l_over = [np.mean(hist2_over.history['accuracy']), np.mean(hist2_over2.history['accuracy']), np.mean(hist2_over3.history['accuracy']),
             np.mean(hist2_over4.history['accuracy'])]

In [201]:
mean_train_l_over = np.mean(accs_train_l_over)
print('over-sampling train accuracy mean after lasso: %.2f%%' % (mean_train_l_over*100))

over-sampling train accuracy mean after lasso: 88.69%


In [202]:
std_train_l_over = np.std(accs_train_l_over)
print('over-sampling train accuracy standard deviation after lasso:', std_train_l_over)

over-sampling train accuracy standard deviation after lasso: 0.009472034
